<center>

# Universidad de Los Andes
## Maestría en Inteligencia Analítica para la toma de decisiones
### Cusro: Supply Chain Analytics

### Sergio Alberto Mora Pardo. Cod.: 201920547

**La solución se encuentra al final del Notebook**

# Locating P Facilities
Make sure you've installed PuLP, Pandas and Plotly

In [2]:
# This is the block you should make changes
# Only change things to the right of the '=' sign. 
# Note that the '#' symbol means a comment-- when the code runs, it ignores this.
# Here you can change the scenario name, number of warehouses, the distance bands, and whether you draw the maps

# ********  To Run, go to the menu item Cell and select 'Run All' *************

# To see the results of the run: 
#      Go to the directory where you put this file and you'll see two output files
#      One output file will be the summary and one will have details
#      And, tabs will open if you decide to map the input and output

# You should change the name to the right of the '=' sign. This names each scenario. 
# If you don't change it, you'll overwrite the files when you run.  Don't put spaces in the name
# **** Change Scenario Name ******  And, keep the single quotes in the name.  

scenario_name = 'scenario_1'  #<<<----- you can change this name

# **** Change the number of warehouses you would like to select.*****  This number should be an integer

number_of_whs = 10  #<<<------ you can change this number

# ********  Change the distance bands **********, if you'd like. The band numbers represent the distance up to.  These should be increasing.
# You cannot add or subtract bands.

distance_band_1 = 100  #<<<----- you can change these numbers
distance_band_2 = 400
distance_band_3 = 800
distance_band_4 = 3200


#************This is where you change the status of a warehouse************
# meaning you can force it to be open or force it closed.
# You want to change the two numbers after the name.  Don't delete or change the commas
# it should be 0,1 to let the solver pick, 1,1 to force it to be used, or 0,0 to force it closed.  1,0 is an error- don't use that

wh_status = {1:('Allentown',0,1),
    2:('Atlanta',0,1),
    3:('Baltimore',0,1),
    4:('Boston',0,1),
    5:('Chicago',0,1),
    6:('Cincinnati',0,1),
    7:('Columbus',0,1),
    8:('Dallas',0,1),
    9:('Denver',0,1),
    10:('Indianapolis',0,1),
    11:('Jacksonville',0,1),
    12:('Kansas City',0,1),
    13:('Las Vegas',0,1),
    14:('Los Angeles',0,1),
    15:('Memphis',0,1),
    16:('Minneapolis',0,1),
    17:('Nashville',0,1),
    18:('New Orleans',0,1),
    19:('Phoenix',0,1),
    20:('Pittsburgh',0,1),
    21:('Raleigh',0,1),
    22:('Reno',0,1),
    23:('San Francisco',0,1),
    24:('Seattle',0,1),
    25:('St. Louis',0,1),
    26:('Lubbock - Current WH',0,1)}

# ****  If you don't want either the input or output map to draw, type False with a capital F instead of True

input_map = True   #<<<----- you can change these from True to False
output_map = True

# ********** To Run, go to the menu item Cell and select 'Run All'**************


In [3]:
#This is code to start running the model.  You don't need to interact with this
from pulp import *
import time
import pandas as pd
import plotly as py


start_time=time.time()

def optimal_location(number_of_whs, warehouses, customers, customer_demands, distance, distance_band, scenario_name):
    # Create the 'prob' variable to contain the problem data
    als_problem = LpProblem("Als", LpMinimize)

    # A dictionary called 'assign_vars' is created to contain the decision variables Yi,j - the assignment of the customer to a facility
    # lower bound is 0; upper bound is 1; integer variable - this makes it binary
    assign_vars = LpVariable.dicts("Asssignment",[(w, c) for w in warehouses for c in customers],0,1,LpInteger)
    # Another dicionary called 'facility_vars' is created to contain the decision variables Xi - the decision of a facility at city i to be opened or not
    facility_vars = LpVariable.dicts("Open", [w for w in warehouses],0,1,LpInteger)

    # The objective function is added to 'prob' first
    total_weighted_demand_objective = lpSum([customer_demands[c]*distance[w,c]*assign_vars[w,c] 
                       for w in warehouses for c in customers])

    # The five constraints are added to 'als_problem'

    # Every customer must be served
    """
    
    A Pulp LP constraint: LpConstraint(e, sense, name, rhs)
    Parameters:	
    e – an instance of LpExpression
    sense – one of LpConstraintEQ, LpConstraintGE, LpConstraintLE
    name – identifying string
    rhs – numerical value of constraint right hand side
    """
    for customer in customers:
        als_problem += LpConstraint(e = lpSum([assign_vars[w, customer] for w in warehouses]), 
                                    sense=LpConstraintEQ, 
                                    name=str(customer)+"_Served", 
                                    rhs=1)

    # Ensure that P and exactly P facilities are opened.  We use the variable name 'number_of_whs' for P
    als_problem += LpConstraint(e = lpSum([facility_vars[w] for w in warehouses]), 
                                    sense=LpConstraintEQ, 
                                    name="FacilityCount", 
                                    rhs=number_of_whs)
    
    # Use the user's status to fix some warehouses in the solution
   
    for facility in warehouses:
        w = facility 
        als_problem += LpConstraint(e = facility_vars[facility] , 
                                   sense=LpConstraintGE, 
                                    name=str(facility) + "_" + "Lower Bound",
                                    rhs=wh_status[w][1])
    for facility in warehouses:
        w = facility
        als_problem += LpConstraint(e = facility_vars[facility] , 
                                   sense=LpConstraintLE, 
                                    name=str(facility) + "_" + "Upper Bound",
                                    rhs=wh_status[w][2])

    # ***************** In this exercise, we'll toggle between these constraints ************** 
    # An assignment can only exist between facility and customer if the facility is opened. We'll call this the "route"
#     for facility in warehouses:
#         for customer in customers:
#             als_problem += LpConstraint(e = assign_vars[facility, customer]-facility_vars[facility] , 
#                                     sense=LpConstraintLE, 
#                                     name=str(facility) + "_" + str(customer) + "_Route",
#                                     rhs=0)
    for facility in warehouses:
        bigM = 1000
        als_problem += LpConstraint(e = lpSum(assign_vars[facility, customer] for customer in customers)-bigM*facility_vars[facility] , 
                                    sense=LpConstraintLE, 
                                    name=str(facility) + "_Route-BigM Version",
                                    rhs=0)        
    

    # Setting problem objective
    als_problem.setObjective(total_weighted_demand_objective)
    # The problem data is written to an .lp file
    #als_problem.writeLP("NineCity.lp")

    # The problem is solved using PuLP's choice of Solver
    _solver = pulp.PULP_CBC_CMD(keepFiles=False,fracGap=0.00,maxSeconds=120, msg=True)
    als_problem.solve(solver=_solver)

    # The status of the solution is printed to the screen
    #print ("Status:", LpStatus[als_problem.status])
    file.write('\nstatus:'+ LpStatus[als_problem.status])
    print("Optimization Status",LpStatus[als_problem.status] ) #print in Jupyter Notebook
    if LpStatus[als_problem.status] == "Infeasible" :
        print("********* ERROR: Model not feasible, don't use results. One thing to check: see that the number of available warehouses is more than the fixed status.")
    #print objective
    total_demand = sum(customer_demands.values())
    total_demand_to_warehouse = {w: sum(customer_demands[c]*assign_vars[w,c].varValue for c in customers)
                                    for w in warehouses
                                    if facility_vars[w].varValue>0}
    
    #print(total_demand_to_warehouse)
    #print("Total Demand",total_demand)
    file.write("\nTotal Demand:"+ str(total_demand))
    #print("Objective: ", value(als_problem.objective))
    file.write("\nObjective: "+ str(value(als_problem.objective)))
    wgt_avg_dist = value(als_problem.objective)/total_demand
    #print("Weighted Average Distance: {:.1f} miles" .format(wgt_avg_dist))
    file.write("\nWeighted Average Distance: {:.1f} miles" .format(wgt_avg_dist))
    print("Weighted Average Distance: {:.1f} miles" .format(wgt_avg_dist))
    
    end_time = time.time()
    time_diff = end_time - start_time
    #print("Run Time in seconds {:.1f}" .format(time_diff))
    file.write("\nRun Time in seconds {:.1f}" .format(time_diff))
    print("Run Time in seconds {:.1f}" .format(time_diff))
 
    
    #preparing data to write in excel sheets
    opened_warehouses = []
      
    for w in facility_vars.keys():
        if(facility_vars[w].varValue > 0):
            #print(warehouses[w])
            wh = {
                'Warehouse Key': w,
                'Warehouse City':warehouses[w][1],
                'State':warehouses[w][2],
                'ZipCode':warehouses[w][3],
                'Lat':warehouses[w][4],
                'Lon':warehouses[w][5],
                'Total Demand to Warehouse':total_demand_to_warehouse[w]
            }
            opened_warehouses.append(wh)
            
            
    #converting the list to dataframe        
    df_wh = pd.DataFrame.from_records(opened_warehouses)
    
    df_wh = df_wh[['Warehouse Key', 'Warehouse City', 'State', 'ZipCode', 'Total Demand to Warehouse']]
    list_warehouses_open = list(df_wh['Warehouse Key'])
    
    # writing detailed files
    writer = pd.ExcelWriter(scenario_name+'_detailed.xlsx')
    df_wh.to_excel(writer,'Opened Warehouses',index=False)
    
#     print("Customers Assigned to warehouses")
    customers_assignment = []   
    for (w,c) in assign_vars.keys():
        if assign_vars[(w,c)].varValue > 0:
            cust = {
                'Warehouse':str(warehouses[w][1]+','+warehouses[w][2]),
                'Customer':str(customers[c][1]+','+customers[c][2]),
                'Customer Demand': customer_demands[c],
                'Distance': distance[w,c],
                'Warehouse Latitude' : warehouses[w][4],
                'Warehouse Longitude' : warehouses[w][5],
                'Customers Latitude' : customers[c][4],
                'Customers Longitude': customers[c][5]
            }
            customers_assignment.append(cust)
                  
    df_cu = pd.DataFrame.from_records(customers_assignment)
    df_cu_copy = df_cu.copy()
    df_cu = df_cu[['Warehouse', 'Customer', 'Distance', 'Customer Demand']]
    df_cu.to_excel(writer,'Customers Assignment',index=False)
    
    
    writer.close()

    #writing percent demand within each distance bands
    total_demand = sum(df_cu['Customer Demand'])
    percent_demand_distance_band_1 = sum(df_cu[df_cu['Distance']<distance_band_1]['Customer Demand'])*100/total_demand
    percent_demand_distance_band_2 = sum(df_cu[df_cu['Distance']<distance_band_2]['Customer Demand'])*100/total_demand
    percent_demand_distance_band_3 = sum(df_cu[df_cu['Distance']<distance_band_3]['Customer Demand'])*100/total_demand
    percent_demand_distance_band_4 = sum(df_cu[df_cu['Distance']<distance_band_4]['Customer Demand'])*100/total_demand
    file.write("\nPercent Demand served within {} miles : {:.1f}" .format(distance_band[0], percent_demand_distance_band_1))
    file.write("\nPercent Demand served within {} miles : {:.1f}" .format(distance_band[1], percent_demand_distance_band_2))
    file.write("\nPercent Demand served within {} miles : {:.1f}" .format(distance_band[2], percent_demand_distance_band_3))
    file.write("\nPercent Demand served within {} miles : {:.1f}" .format(distance_band[3], percent_demand_distance_band_4))
    
    print("Percent Demand served within {} miles : {:.1f}" .format(distance_band[0], percent_demand_distance_band_1))
    print("Percent Demand served within {} miles : {:.1f}" .format(distance_band[1], percent_demand_distance_band_2))
    print("Percent Demand served within {} miles : {:.1f}" .format(distance_band[2], percent_demand_distance_band_3))
    print("Percent Demand served within {} miles : {:.1f}" .format(distance_band[3], percent_demand_distance_band_4))
   
    return df_cu_copy,list_warehouses_open,df_wh
         
            


Academic license - for non-commercial use only
No parameters matching '_test' found


In [4]:
#  This is the input data.  In a normal project, you would load this from another file.  We are making it simple by packaging it here 
def get_data():
    # Warehouses:: 
    # id:(name, city, state, zipcode, lat, long)
    warehouses = {1:('Allentown','Allentown','PA','18101',40.602812,-75.470433),
    2:('Atlanta','Atlanta','GA','30301',33.753693,-84.389544),
    3:('Baltimore','Baltimore','MD','21201',39.294398,-76.622747),
    4:('Boston','Boston','MA','02101',42.36097,-71.05344),
    5:('Chicago','Chicago','IL','60602',41.88331,-87.624713),
    6:('Cincinnati','Cincinnati','OH','45201',39.10663,-84.49974),
    7:('Columbus','Columbus','OH','43201',39.991395,-83.001036),
    8:('Dallas','Dallas','TX','75201',32.787642,-96.799525),
    9:('Denver','Denver','CO','80201',39.75071,-104.996225),
    10:('Indianapolis','Indianapolis','IN','46201',39.77422,-86.109309),
    11:('Jacksonville','Jacksonville','FL','32201',30.32769,-81.64815),
    12:('Kansas City','Kansas City','MO','64101',39.103883,-94.600613),
    13:('Las Vegas','Las Vegas','NV','89101',36.17269,-115.121117),
    14:('Los Angeles','Los Angeles','CA','90001',33.974044,-118.248849),
    15:('Memphis','Memphis','TN','37501',35.033731,-89.934319),
    16:('Minneapolis','Minneapolis','MN','55401',44.985775,-93.270165),
    17:('Nashville','Nashville','TN','37201',36.164003,-86.7745),
    18:('New Orleans','New Orleans','LA','70112',29.956664,-90.077506),
    19:('Phoenix','Phoenix','AZ','85001',33.451015,-112.068554),
    20:('Pittsburgh','Pittsburgh','PA','15201',40.474802,-79.95449),
    21:('Raleigh','Raleigh','NC','27601',35.773856,-78.634051),
    22:('Reno','Reno','NV','89501',39.526866,-119.811392),
    23:('San Francisco','San Francisco','CA','94102',37.779887,-122.418066),
    24:('Seattle','Seattle','WA','98101',47.611601,-122.333038),
    25:('St. Louis','St. Louis','MO','63101',38.631358,-90.192246),
    26:('Lubbock - Current WH','Lubbock','TX','79401',33.584601,-101.846676)}

    # customers:: 
    # id:(name, city, state, zipcode, lat, long)
    customers = {1:('Akron','Akron','OH','  ',41.08,-81.52),
    2:('Albuquerque','Albuquerque','NM','  ',35.12,-106.62),
    3:('Alexandria','Alexandria','VA','  ',38.82,-77.09),
    4:('Amarillo','Amarillo','TX','  ',35.2,-101.82),
    5:('Anaheim','Anaheim','CA','  ',33.84,-117.87),
    6:('Brownfield','Brownfield','TX','  ',33.18101,-102.27066),
    7:('Arlington','Arlington','TX','  ',32.69,-97.13),
    8:('Arlington','Arlington','VA','  ',38.88,-77.1),
    9:('Atlanta','Atlanta','GA','  ',33.76,-84.42),
    10:('Augusta-Richmond','Augusta-Richmond','GA','  ',33.46,-81.99),
    11:('Aurora','Aurora','CO','  ',39.71,-104.73),
    12:('Aurora','Aurora','IL','  ',41.77,-88.29),
    13:('Austin','Austin','TX','  ',30.31,-97.75),
    14:('Bakersfield','Bakersfield','CA','  ',35.36,-119),
    15:('Baltimore','Baltimore','MD','  ',39.3,-76.61),
    16:('Baton Rouge','Baton Rouge','LA','  ',30.45,-91.13),
    17:('Bellevue','Bellevue','WA','  ',47.6,-122.16),
    18:('Birmingham','Birmingham','AL','  ',33.53,-86.8),
    19:('Boise City','Boise City','ID','  ',43.61,-116.23),
    20:('Boston','Boston','MA','  ',42.34,-71.02),
    21:('Bridgeport','Bridgeport','CT','  ',41.19,-73.2),
    22:('Brownsville','Brownsville','TX','  ',25.93,-97.48),
    23:('Buffalo','Buffalo','NY','  ',42.89,-78.86),
    24:('Cape Coral','Cape Coral','FL','  ',26.64,-82),
    25:('Carrollton','Carrollton','TX','  ',32.99,-96.9),
    26:('Cary','Cary','NC','  ',35.78,-78.8),
    27:('Cedar Rapids','Cedar Rapids','IA','  ',41.97,-91.67),
    28:('Chandler','Chandler','AZ','  ',33.3,-111.87),
    29:('Charlotte','Charlotte','NC','  ',35.2,-80.83),
    30:('Chattanooga','Chattanooga','TN','  ',35.07,-85.26),
    31:('Chesapeake','Chesapeake','VA','  ',36.68,-76.31),
    32:('Chicago','Chicago','IL','  ',41.84,-87.68),
    33:('Chula Vista','Chula Vista','CA','  ',32.63,-117.04),
    34:('Cincinnati','Cincinnati','OH','  ',39.14,-84.51),
    35:('Cleveland','Cleveland','OH','  ',41.48,-81.68),
    36:('Colorado Springs','Colorado Springs','CO','  ',38.86,-104.76),
    37:('Columbia','Columbia','SC','  ',34.04,-80.89),
    38:('Columbus','Columbus','OH','  ',39.99,-82.99),
    39:('Columbus','Columbus','GA','  ',32.51,-84.87),
    40:('Corona','Corona','CA','  ',33.87,-117.57),
    41:('Corpus Christi','Corpus Christi','TX','  ',27.71,-97.29),
    42:('Dallas','Dallas','TX','  ',32.79,-96.77),
    43:('Dayton','Dayton','OH','  ',39.78,-84.2),
    44:('Denton','Denton','TX','  ',33.21,-97.13),
    45:('Denver','Denver','CO','  ',39.77,-104.87),
    46:('Des Moines','Des Moines','IA','  ',41.58,-93.62),
    47:('Detroit','Detroit','MI','  ',42.38,-83.1),
    48:('Durham','Durham','NC','  ',35.98,-78.91),
    49:('East Los Angeles','East Los Angeles','CA','  ',34.03,-118.17),
    50:('Elk Grove','Elk Grove','CA','  ',38.4,-121.37),
    51:('El Paso','El Paso','TX','  ',31.85,-106.44),
    52:('Escondido','Escondido','CA','  ',33.14,-117.07),
    53:('Eugene','Eugene','OR','  ',44.05,-123.11),
    54:('Fayetteville','Fayetteville','NC','  ',35.07,-78.9),
    55:('Fontana','Fontana','CA','  ',34.1,-117.46),
    56:('Fort Collins','Fort Collins','CO','  ',40.56,-105.07),
    57:('Fort Lauderdale','Fort Lauderdale','FL','  ',26.14,-80.14),
    58:('Fort Wayne','Fort Wayne','IN','  ',41.07,-85.14),
    59:('Fort Worth','Fort Worth','TX','  ',32.75,-97.34),
    60:('Fremont','Fremont','CA','  ',37.53,-122),
    61:('Fresno','Fresno','CA','  ',36.78,-119.79),
    62:('Fullerton','Fullerton','CA','  ',33.88,-117.93),
    63:('Garden Grove','Garden Grove','CA','  ',33.78,-117.96),
    64:('Garland','Garland','TX','  ',32.91,-96.63),
    65:('Gilbert','Gilbert','AZ','  ',33.33,-111.76),
    66:('Glendale','Glendale','AZ','  ',33.58,-112.2),
    67:('Glendale','Glendale','CA','  ',34.18,-118.25),
    68:('Grand Prairie','Grand Prairie','TX','  ',32.69,-97.02),
    69:('Grand Rapids','Grand Rapids','MI','  ',42.96,-85.66),
    70:('Greensboro','Greensboro','NC','  ',36.08,-79.83),
    71:('Hampton','Hampton','VA','  ',37.05,-76.29),
    72:('Hayward','Hayward','CA','  ',37.63,-122.1),
    73:('Henderson','Henderson','NV','  ',36.03,-115),
    74:('Hialeah','Hialeah','FL','  ',25.86,-80.3),
    75:('Highlands Ranch','Highlands Ranch','CO','  ',39.55,-104.97),
    76:('Hollywood','Hollywood','FL','  ',26.03,-80.16),
    77:('Concord','Concord','NH','  ',43.2347496666667,-71.5435606666667),
    78:('Houston','Houston','TX','  ',29.77,-95.39),
    79:('Huntington Beach','Huntington Beach','CA','  ',33.69,-118.01),
    80:('Huntsville','Huntsville','AL','  ',34.71,-86.63),
    81:('Indianapolis','Indianapolis','IN','  ',39.78,-86.15),
    82:('Irvine','Irvine','CA','  ',33.66,-117.8),
    83:('Irving','Irving','TX','  ',32.86,-96.97),
    84:('Jackson','Jackson','MS','  ',32.32,-90.21),
    85:('Jacksonville','Jacksonville','FL','  ',30.33,-81.66),
    86:('Jersey City','Jersey City','NJ','  ',40.71,-74.06),
    87:('Joliet','Joliet','IL','  ',41.53,-88.12),
    88:('Kansas City','Kansas City','MO','  ',39.12,-94.55),
    89:('Kansas City','Kansas City','KS','  ',39.12,-94.73),
    90:('Knoxville','Knoxville','TN','  ',35.97,-83.95),
    91:('Lakewood','Lakewood','CO','  ',39.7,-105.11),
    92:('Lancaster','Lancaster','CA','  ',34.69,-118.18),
    93:('Laredo','Laredo','TX','  ',27.53,-99.49),
    94:('Las Vegas','Las Vegas','NV','  ',36.21,-115.22),
    95:('Lexington','Lexington','KY','  ',38.04,-84.46),
    96:('Lincoln','Lincoln','NE','  ',40.82,-96.69),
    97:('Little Rock','Little Rock','AR','  ',34.72,-92.35),
    98:('Long Beach','Long Beach','CA','  ',33.79,-118.16),
    99:('Los Angeles','Los Angeles','CA','  ',34.11,-118.41),
    100:('Louisville','Louisville','KY','  ',38.22,-85.74),
    101:('Lubbock','Lubbock','TX','  ',33.58,-101.88),
    102:('MacAllen','MacAllen','TX','  ',26.22,-98.24),
    103:('MacKinney','MacKinney','TX','  ',33.2,-96.65),
    104:('Madison','Madison','WI','  ',43.08,-89.39),
    105:('Memphis','Memphis','TN','  ',35.11,-90.01),
    106:('Mesa','Mesa','AZ','  ',33.42,-111.74),
    107:('Mesquite','Mesquite','TX','  ',32.77,-96.6),
    108:('Metairie','Metairie','LA','  ',30,-90.18),
    109:('Miami','Miami','FL','  ',25.78,-80.21),
    110:('Milwaukee','Milwaukee','WI','  ',43.06,-87.97),
    111:('Minneapolis','Minneapolis','MN','  ',44.96,-93.27),
    112:('Mobile','Mobile','AL','  ',30.68,-88.09),
    113:('Modesto','Modesto','CA','  ',37.66,-120.99),
    114:('Montgomery','Montgomery','AL','  ',32.35,-86.28),
    115:('Moreno Valley','Moreno Valley','CA','  ',33.93,-117.21),
    116:('Naperville','Naperville','IL','  ',41.76,-88.15),
    117:('Nashville','Nashville','TN','  ',36.17,-86.78),
    118:('Newark','Newark','NJ','  ',40.72,-74.17),
    119:('New Orleans','New Orleans','LA','  ',30.07,-89.93),
    120:('Newport News','Newport News','VA','  ',37.08,-76.51),
    121:('New York','New York','NY','  ',40.67,-73.94),
    122:('Norfolk','Norfolk','VA','  ',36.92,-76.24),
    123:('North Las Vegas','North Las Vegas','NV','  ',36.27,-115.14),
    124:('Oakland','Oakland','CA','  ',37.77,-122.22),
    125:('Oceanside','Oceanside','CA','  ',33.23,-117.31),
    126:('Oklahoma City','Oklahoma City','OK','  ',35.47,-97.51),
    127:('Omaha','Omaha','NE','  ',41.26,-96.01),
    128:('Ontario','Ontario','CA','  ',34.05,-117.61),
    129:('Orange','Orange','CA','  ',33.81,-117.82),
    130:('Orlando','Orlando','FL','  ',28.5,-81.37),
    131:('Overland Park','Overland Park','KS','  ',38.91,-94.68),
    132:('Oxnard','Oxnard','CA','  ',34.2,-119.21),
    133:('Palmdale','Palmdale','CA','  ',34.61,-118.09),
    134:('Paradise','Paradise','NV','  ',36.08,-115.13),
    135:('Pasadena','Pasadena','TX','  ',29.66,-95.15),
    136:('Pasadena','Pasadena','CA','  ',34.16,-118.14),
    137:('Paterson','Paterson','NJ','  ',40.91,-74.16),
    138:('Pembroke Pines','Pembroke Pines','FL','  ',26.01,-80.34),
    139:('San Angelo','San Angelo','TX','  ',31.44322,-100.46333),
    140:('Philadelphia','Philadelphia','PA','  ',40.01,-75.13),
    141:('Phoenix','Phoenix','AZ','  ',33.54,-112.07),
    142:('Pittsburgh','Pittsburgh','PA','  ',40.44,-79.98),
    143:('Plano','Plano','TX','  ',33.05,-96.75),
    144:('Pomona','Pomona','CA','  ',34.06,-117.76),
    145:('Portland','Portland','OR','  ',45.54,-122.66),
    146:('Port Saint Lucie','Port Saint Lucie','FL','  ',27.28,-80.35),
    147:('Providence','Providence','RI','  ',41.82,-71.42),
    148:('Raleigh','Raleigh','NC','  ',35.82,-78.66),
    149:('Del Rio','Del Rio','TX','  ',29.36041,-100.89904),
    150:('Reno','Reno','NV','  ',39.54,-119.82),
    151:('Richmond','Richmond','VA','  ',37.53,-77.47),
    152:('Riverside','Riverside','CA','  ',33.94,-117.4),
    153:('Rochester','Rochester','NY','  ',43.17,-77.62),
    154:('Rockford','Rockford','IL','  ',42.27,-89.06),
    155:('Sacramento','Sacramento','CA','  ',38.57,-121.47),
    156:('Saint Louis','Saint Louis','MO','  ',38.64,-90.24),
    157:('Saint Paul','Saint Paul','MN','  ',44.95,-93.1),
    158:('Saint Petersburg','Saint Petersburg','FL','  ',27.76,-82.64),
    159:('Salem','Salem','OR','  ',44.92,-123.02),
    160:('Salinas','Salinas','CA','  ',36.68,-121.64),
    161:('Salt Lake City','Salt Lake City','UT','  ',40.78,-111.93),
    162:('San Antonio','San Antonio','TX','  ',29.46,-98.51),
    163:('San Bernardino','San Bernardino','CA','  ',34.14,-117.29),
    164:('San Diego','San Diego','CA','  ',32.81,-117.14),
    165:('San Francisco','San Francisco','CA','  ',37.77,-122.45),
    166:('San Jose','San Jose','CA','  ',37.3,-121.85),
    167:('Santa Ana','Santa Ana','CA','  ',33.74,-117.88),
    168:('Santa Clarita','Santa Clarita','CA','  ',34.41,-118.51),
    169:('Santa Rosa','Santa Rosa','CA','  ',38.45,-122.7),
    170:('Savannah','Savannah','GA','  ',32.02,-81.13),
    171:('Scottsdale','Scottsdale','AZ','  ',33.69,-111.87),
    172:('Seattle','Seattle','WA','  ',47.62,-122.35),
    173:('Shreveport','Shreveport','LA','  ',32.47,-93.8),
    174:('Sioux Falls','Sioux Falls','SD','  ',43.54,-96.73),
    175:('Spokane','Spokane','WA','  ',47.67,-117.41),
    176:('Springfield','Springfield','MO','  ',37.2,-93.29),
    177:('Springfield','Springfield','MA','  ',42.12,-72.54),
    178:('Spring Valley','Spring Valley','NV','  ',36.11,-115.24),
    179:('Stockton','Stockton','CA','  ',37.97,-121.31),
    180:('Sunnyvale','Sunnyvale','CA','  ',37.39,-122.03),
    181:('Sunrise Manor','Sunrise Manor','NV','  ',36.19,-115.05),
    182:('Syracuse','Syracuse','NY','  ',43.04,-76.14),
    183:('Tacoma','Tacoma','WA','  ',47.25,-122.46),
    184:('Tallahassee','Tallahassee','FL','  ',30.46,-84.28),
    185:('Tampa','Tampa','FL','  ',27.96,-82.48),
    186:('Tempe','Tempe','AZ','  ',33.39,-111.93),
    187:('Toledo','Toledo','OH','  ',41.66,-83.58),
    188:('Toms River','Toms River','NJ','  ',39.94,-74.17),
    189:('Torrance','Torrance','CA','  ',33.83,-118.34),
    190:('Tucson','Tucson','AZ','  ',32.2,-110.89),
    191:('Tulsa','Tulsa','OK','  ',36.13,-95.92),
    192:('Vancouver','Vancouver','WA','  ',45.63,-122.64),
    193:('Virginia Beach','Virginia Beach','VA','  ',36.74,-76.04),
    194:('Visalia','Visalia','CA','  ',36.33,-119.32),
    195:('Warren','Warren','MI','  ',42.49,-83.03),
    196:('Washington','Washington','DC','  ',38.91,-77.02),
    197:('Wichita','Wichita','KS','  ',37.69,-97.34),
    198:('Winston-Salem','Winston-Salem','NC','  ',36.1,-80.26),
    199:('Worcester','Worcester','MA','  ',42.27,-71.81),
    200:('Yonkers','Yonkers','NY','  ',40.95,-73.87)}
    #  demand::
    # id: demand_value
    customer_demands = {1:205375,2:535923,3:147786,4:190449,5:342336,6:283690,7:385024,8:214890,9:571861,10:199489,
                        11:334924,12:176710,13:792778,14:336429,15:632410,16:221091,17:128225,18:226152,19:210099,
                        20:610407,21:136604,22:183598,23:267037,24:161428,25:130013,26:145939,27:130349,28:258122,
                        29:723514,30:173001,31:223315,32:2878948,33:226606,34:332807,35:424526,36:388306,37:130246,
                        38:768662,39:193007,40:152977,41:296937,42:1304930,43:151594,44:127364,45:609651,46:200202,
                        47:901160,48:235337,49:135682,50:141026,51:631862,52:139583,53:152842,54:177632,55:191526,
                        56:141921,57:184625,58:253250,59:751149,60:207431,61:490930,62:133165,63:167800,64:220597,
                        65:243090,66:255662,67:199608,68:167767,69:193337,70:259523,71:147560,72:145517,73:265681,
                        74:209524,75:129817,76:142105,77:375134,78:2307883,79:194227,80:185501,81:803930,82:223785,
                        83:206798,84:171210,85:822401,86:242594,87:153577,88:454520,89:144431,90:187071,91:142294,
                        92:151452,93:232298,94:570083,95:279146,96:257612,97:192247,98:464505,99:3878715,100:556482,
                        101:226180,102:136555,103:135935,104:237838,105:662989,106:474115,107:134509,108:141919,
                        109:425242,110:606508,111:386751,112:190007,113:203882,114:200848,115:199353,116:146177,
                        117:605197,118:279278,119:206720,120:182057,121:8459026,122:234477,123:236574,124:412612,
                        125:172679,126:564147,127:449283,128:174731,129:141351,130:235745,131:176161,132:189413,
                        133:150264,134:261122,135:148101,136:144485,137:145162,138:146408,139:168424,140:1445993,
                        141:1635783,142:305708,143:282016,144:154638,145:573530,146:163859,147:170868,148:421938,
                        149:176424,150:221051,151:198179,152:302653,153:205236,154:159109,155:475452,156:349763,
                        157:281501,158:243829,159:157890,160:145628,161:184856,162:1402013,163:199314,164:1309749,
                        165:817411,166:977893,167:341991,168:171722,169:159730,170:133244,171:241464,172:614214,
                        173:204311,174:160938,175:204424,176:158605,177:149926,178:212144,179:291422,180:135564,
                        181:219092,182:136410,183:200855,184:175438,185:351091,186:181658,187:288207,188:131619,
                        189:141478,190:551067,191:387683,192:169603,193:439564,194:127444,195:133470,196:597885,
                        197:374803,198:223127,199:174854,200:206441}
    # (from, to): distance
    distance = {(1,1):374,(1,2):2041,(1,3):177,(1,4):1742,(1,5):2777,(1,7):1550,(1,8):173,(1,9):804,(1,10):719,
                (1,11):1815,(1,12):791,(1,13):1692,(1,14):2799,(1,15):127,(1,16):1323,(1,17):2747,(1,18):933,
                (1,19):2451,(1,20):306,(1,21):147,(1,6):1839,(1,22):1907,(1,23):277,(1,24):1219,(1,25):1524,
                (1,26):446,(1,27):996,(1,28):2424,(1,29):558,(1,30):773,(1,31):324,(1,32):754,(1,33):2767,(1,34):577,
                (1,35):388,(1,36):1832,(1,37):639,(1,38):469,(1,39):900,(1,77):320,(1,40):2758,(1,41):1799,(1,42):1524,
                (1,43):547,(1,149):1918,(1,44):1528,(1,45):1822,(1,46):1115,(1,47):487,(1,48):436,(1,49):2790,(1,51):2143,
                (1,50):2858,(1,52):2752,(1,53):2843,(1,54):501,(1,55):2743,(1,56):1822,(1,57):1219,(1,58):596,(1,59):1560,
                (1,60):2919,(1,61):2805,(1,62):2780,(1,63):2785,(1,64):1511,(1,65):2417,(1,66):2435,(1,67):2790,(1,68):1543,
                (1,69):647,(1,70):461,(1,71):294,(1,72):2923,(1,73):2532,(1,74):1244,(1,75):1832,(1,76):1228,(1,78):1587,
                (1,79):2791,(1,80):864,(1,81):667,(1,82):2779,(1,83):1533,(1,84):1175,(1,85):931,(1,86):87,(1,87):779,
                (1,88):1196,(1,89):1208,(1,90):660,(1,91):1838,(1,92):2769,(1,93):1929,(1,94):2541,(1,95):603,(1,96):1307,
                (1,97):1186,(1,98):2797,(1,99):2802,(1,100):674,(1,101):1801,(1,102):1931,(1,103):1500,(1,104):867,
                (1,105):1035,(1,106):2412,(1,107):1515,(1,108):1301,(1,109):1249,(1,110):783,(1,111):1119,(1,112):1160,
                (1,113):2854,(1,114):975,(1,115):2734,(1,116):782,(1,117):806,(1,119):1285,(1,121):94,(1,118):80,
                (1,120):294,(1,122):304,(1,123):2534,(1,124):2926,(1,125):2763,(1,126):1470,(1,127):1262,(1,128):2754,
                (1,129):2775,(1,130):1061,(1,131):1208,(1,132):2848,(1,133):2766,(1,134):2539,(1,135):1579,(1,136):2783,(1,137):84,(1,138):1233,(1,140):52,(1,141):2429,(1,142):279,(1,143):1512,(1,144):2763,(1,146):1133,(1,145):2796,(1,147):267,(1,148):439,(1,150):2736,(1,151):280,(1,152):2745,(1,153):246,(1,154):840,(1,155):2859,(1,156):939,(1,157):1109,(1,158):1151,(1,159):2824,(1,160):2921,(1,161):2235,(1,139):1799,(1,162):1776,(1,163):2732,(1,164):2767,(1,165):2940,(1,166):2916,(1,167):2781,(1,168):2798,(1,169):2937,(1,170):791,(1,171):2411,(1,172):2757,(1,173):1366,(1,174):1304,(1,175):2486,(1,178):2545,(1,176):1161,(1,177):217,(1,179):2865,(1,180):2925,(1,181):2531,(1,182):202,(1,183):2766,(1,184):1010,(1,185):1132,(1,186):2425,(1,187):504,(1,188):97,(1,189):2807,(1,190):2403,(1,191):1352,(1,192):2793,(1,193):316,(1,194):2789,(1,195):485,(1,196):168,(1,197):1398,(1,198):477,(1,199):261,(1,200):102,(2,1):625,(2,2):1494,(2,3):632,(2,4):1175,(2,5):2256,(2,7):872,(2,8):634,(2,9):2,(2,10):164,(2,11):1410,(2,12):699,(2,13):963,(2,14):2314,(2,15):680,(2,16):537,(2,17):2558,(2,18):164,(2,19):2162,(2,20):1104,(2,21):942,(2,6):1214,(2,22):1121,(2,23):823,(2,24):603,(2,25):852,(2,26):408,(2,27):816,(2,28):1861,(2,29):266,(2,30):122,(2,31):587,(2,32):691,(2,33):2218,(2,34):438,(2,35):653,(2,36):1397,(2,37):237,(2,38):516,(2,39):106,(2,77):1123,(2,40):2236,(2,41):1027,(2,42):846,(2,43):491,(2,149):1199,(2,44):866,(2,45):1419,(2,46):871,(2,47):707,(2,48):408,(2,49):2274,(2,51):1515,(2,50):2447,(2,52):2212,(2,53):2567,(2,54):384,(2,55):2225,(2,56):1448,(2,57):688,(2,58):598,(2,59):885,(2,60):2492,(2,61):2354,(2,62):2260,(2,63):2263,(2,64):835,(2,65):1853,(2,66):1880,(2,67):2277,(2,68):864,(2,69):754,(2,70):358,(2,71):600,(2,72):2498,(2,73):2045,(2,74):704,(2,75):1421,(2,76):696,(2,78):827,(2,79):2267,(2,80):169,(2,81):504,(2,82):2254,(2,83):859,(2,84):414,(2,85):336,(2,86):876,(2,87):677,(2,88):795,(2,89):805,(2,90):182,(2,91):1433,(2,92):2266,(2,93):1171,(2,94):2059,(2,95):349,(2,96):981,(2,97):541,(2,98):2276,(2,99):2289,(2,100):374,(2,101):1184,(2,102):1152,(2,103):833,(2,104):823,(2,105):393,(2,106):1851,(2,107):835,(2,108):503,(2,109):713,(2,110):791,(2,111):1069,(2,112):357,(2,113):2426,(2,114):172,(2,115):2211,(2,116):695,(2,117):253,(2,119):486,(2,121):880,(2,118):871,(2,120):588,(2,122):599,(2,123):2053,(2,124):2505,(2,125):2227,(2,126):890,(2,127):967,(2,128):2236,(2,129):2253,(2,130):476,(2,131):794,(2,132):2341,(2,133):2261,(2,134):2053,(2,135):816,(2,136):2270,(2,137):881,(2,138):692,(2,140):789,(2,141):1871,(2,142):615,(2,143):842,(2,144):2246,(2,146):598,(2,145):2556,(2,147):1060,(2,148):418,(2,150):2347,(2,151):551,(2,152):2223,(2,153):879,(2,154):755,(2,155):2453,(2,156):553,(2,157):1063,(2,158):503,(2,159):2569,(2,160):2475,(2,161):1866,(2,139):1118,(2,162):1039,(2,163):2214,(2,164):2222,(2,165):2520,(2,166):2484,(2,167):2258,(2,168):2292,(2,169):2532,(2,170):263,(2,171):1856,(2,172):2569,(2,173):650,(2,174):1116,(2,175):2308,(2,178):2060,(2,176):653,(2,177):1020,(2,179):2445,(2,180):2495,(2,181):2048,(2,182):920,(2,183):2569,(2,184):268,(2,185):490,(2,186):1864,(2,187):646,(2,188):834,(2,189):2288,(2,190):1810,(2,191):793,(2,192):2556,(2,193):606,(2,194):2326,(2,195):717,(2,196):640,(2,197):913,(2,198):335,(2,199):1062,(2,200):897,(3,1):337,(3,2):1966,(3,3):48,(3,4):1662,(3,5):2711,(3,7):1450,(3,8):45,(3,9):680,(3,10):591,(3,11):1760,(3,12):749,(3,13):1586,(3,14):2739,(3,15):0,(3,16):1205,(3,17):2735,(3,18):815,(3,19):2420,(3,20):425,(3,21):262,(3,6):1750,(3,22):1790,(3,23):323,(3,24):1093,(3,25):1425,(3,26):318,(3,27):954,(3,28):2349,(3,29):430,(3,30):657,(3,31):213,(3,32):714,(3,33):2696,(3,34):497,(3,35):360,(3,36):1772,(3,37):510,(3,38):403,(3,39):775,(3,77):446,(3,40):2691,(3,41):1685,(3,42):1425,(3,43):477,(3,149):1814,(3,44):1431,(3,45):1768,(3,46):1068,(3,47):471,(3,48):307,(3,49):2724,(3,51):2056,(3,50):2810,(3,52):2682,(3,53):2817,(3,54):374,(3,55):2678,(3,56):1772,(3,57):1098,(3,58):549,(3,59):1461,(3,60):2869,(3,61):2751,(3,62):2714,(3,63):2719,(3,64):1412,(3,65):2342,(3,66):2362,(3,67):2725,(3,68):1444,(3,69):629,(3,70):333,(3,71):184,(3,72):2873,(3,73):2470,(3,74):1122,(3,75):1776,(3,76):1107,(3,78):1474,(3,79):2724,(3,80):749,(3,81):599,(3,82):2712,(3,83):1434,(3,84):1060,(3,85):803,(3,86):197,(3,87):735,(3,88):1129,(3,89):1141,(3,90):544,(3,91):1783,(3,92):2706,(3,93):1818,(3,94):2480,(3,95):508,(3,96):1254,(3,97):1087,(3,98):2731,(3,99):2737,(3,100):585,(3,101):1713,(3,102):1816,(3,103):1402,(3,104):840,(3,105):932,(3,106):2338,(3,107):1415,(3,108):1180,(3,109):1128,(3,110):759,(3,111):1104,(3,112):1036,(3,113):2804,(3,114):851,(3,115):2667,(3,116):740,(3,117):701,(3,119):1163,(3,121):201,(3,118):192,(3,120):180,(3,122):194,(3,123):2473,(3,124):2877,(3,125):2695,(3,126):1384,(3,127):1213,(3,128):2689,(3,129):2709,(3,130):935,(3,131):1140,(3,132):2785,(3,133):2702,(3,134):2477,(3,135):1466,(3,136):2719,(3,137):202,(3,138):1111,(3,140):110,(3,141):2355,(3,142):229,(3,143):1414,(3,144):2698,(3,146):1010,(3,145):2776,(3,147):382,(3,148):312,(3,150):2692,(3,151):153,(3,152):2679,(3,153):321,(3,154):803,(3,155):2813,(3,156):863,(3,157):1094,(3,158):1024,(3,159):2802,(3,160):2868,(3,161):2190,(3,139):1701,(3,162):1669,(3,163):2666,(3,164):2697,(3,165):2891,(3,166):2866,(3,167):2715,(3,168):2735,(3,169):2891,(3,170):663,(3,171):2338,(3,172):2745,(3,173):1260,(3,174):1272,(3,175):2474,(3,178):2483,(3,176):1078,(3,177):341,(3,179):2816,(3,180):2875,(3,181):2470,(3,182):306,(3,183):2753,(3,184):882,(3,185):1004,(3,186):2350,(3,187):472,(3,188):162,(3,189):2741,(3,190):2323,(3,191):1267,(3,192):2774,(3,193):211,(3,194):2733,(3,195):472,(3,196):40,(3,197):1324,(3,198):350,(3,199):383,(3,200):218,(4,1):644,(4,2):2317,(4,3):471,(4,4):2026,(4,5):3040,(4,7):1850,(4,8):469,(4,9):1105,(4,10):1008,(4,11):2067,(4,12):1041,(4,13):1996,(4,14):3053,(4,15):423,(4,16):1628,(4,17):2921,(4,18):1238,(4,19):2660,(4,20):2,(4,21):161,(4,6):2130,(4,22):2212,(4,23):469,(4,24):1473,(4,25):1823,(4,26):725,(4,27):1242,(4,28):2700,(4,29):850,(4,30):1078,(4,31):568,(4,32):1004,(4,33):3037,(4,34):869,(4,35):647,(4,36):2090,(4,37):924,(4,38):756,(4,39):1198,(4,77):76,(4,40):3021,(4,41):2104,(4,42):1824,(4,43):833,(4,149):2220,(4,44):1827,(4,45):2074,(4,46):1364,(4,47):724,(4,48):718,(4,49):3051,(4,51):2432,(4,50):3093,(4,52):3019,(4,53):3042,(4,54):774,(4,55):3006,(4,56):2068,(4,57):1454,(4,58):862,(4,59):1859,(4,60):3158,(4,61):3052,(4,62):3042,(4,63):3048,(4,64):1811,(4,65):2692,(4,66):2709,(4,67):3051,(4,68):1843,(4,69):875,(4,70):753,(4,71):542,(4,72):3161,(4,73):2790,(4,74):1479,(4,75):2085,(4,76):1463,(4,78):1892,(4,79):3054,(4,80):1169,(4,81):949,(4,82):3043,(4,83):1833,(4,84):1480,(4,85):1200,(4,86):227,(4,87):1034,(4,88):1469,(4,89):1480,(4,90):965,(4,91):2090,(4,92):3028,(4,93):2234,(4,94):2797,(4,95):904,(4,96):1561,(4,97):1487,(4,98):3060,(4,99):3063,(4,100):971,(4,101):2091,(4,102):2237,(4,103):1800,(4,104):1096,(4,105):1337,(4,106):2687,(4,107):1815,(4,108):1604,(4,109):1483,(4,110):1012,(4,111):1322,(4,112):1461,(4,113):3094,(4,114):1276,(4,115):2997,(4,116):1033,(4,117):1110,(4,119):1588,(4,121):223,(4,118):232,(4,120):549,(4,122):549,(4,123):2790,(4,124):3163,(4,125):3030,(4,126):1760,(4,127):1512,(4,128):3017,(4,129):3038,(4,130):1318,(4,131):1482,(4,132):3108,(4,133):3025,(4,134):2796,(4,135):1884,(4,136):3045,(4,137):223,(4,138):1469,(4,140):314,(4,141):2703,(4,142):567,(4,143):1812,(4,144):3026,(4,146):1374,(4,145):2985,(4,147):49,(4,148):717,(4,150):2968,(4,151):561,(4,152):3008,(4,153):398,(4,154):1082,(4,155):3094,(4,156):1223,(4,157):1312,(4,158):1415,(4,159):3018,(4,160):3165,(4,161):2470,(4,139):2098,(4,162):2080,(4,163):2994,(4,164):3036,(4,165):3177,(4,166):3157,(4,167):3045,(4,168):3058,(4,169):3170,(4,170):1064,(4,171):2685,(4,172):2931,(4,173):1670,(4,174):1528,(4,175):2663,(4,178):2802,(4,176):1449,(4,177):91,(4,179):3103,(4,180):3165,(4,181):2788,(4,182):309,(4,183):2943,(4,184):1297,(4,185):1395,(4,186):2700,(4,187):759,(4,188):274,(4,189):3069,(4,190):2684,(4,191):1642,(4,192):2982,(4,193):554,(4,194):3039,(4,195):719,(4,196):463,(4,197):1676,(4,198):772,(4,199):46,(4,200):206,(5,1):378,(5,2):1326,(5,3):699,(5,4):1053,(5,5):2040,(5,8):696,(5,7):968,(5,9):693,(5,10):776,(5,11):1067,(5,12):41,(5,13):1152,(5,14):2054,(5,15):712,(5,16):959,(5,17):2034,(5,18):682,(5,19):1708,(5,20):1002,(5,21):880,(5,6):1179,(5,22):1458,(5,23):533,(5,24):1297,(5,25):939,(5,26):748,(5,27):245,(5,28):1706,(5,29):695,(5,30):575,(5,31):828,(5,32):4,(5,33):2038,(5,34):295,(5,35):363,(5,36):1089,(5,37):771,(5,38):324,(5,39):784,(5,77):969,(5,40):2021,(5,41):1321,(5,42):947,(5,43):271,(5,149):1343,(5,44):934,(5,45):1075,(5,46):365,(5,47):276,(5,48):731,(5,49):2051,(5,51):1468,(5,50):2107,(5,52):2019,(5,53):2106,(5,54):785,(5,55):2006,(5,56):1072,(5,57):1377,(5,58):165,(5,59):973,(5,60):2168,(5,61):2056,(5,62):2042,(5,63):2048,(5,64):933,(5,65):1698,(5,66):1714,(5,67):2050,(5,68):964,(5,69):147,(5,70):683,(5,71):813,(5,72):2172,(5,73):1790,(5,74):1394,(5,75):1085,(5,76):1385,(5,78):1111,(5,79):2054,(5,80):587,(5,81):193,(5,82):2043,(5,83):950,(5,84):796,(5,85):1019,(5,86):834,(5,87):41,(5,88):484,(5,89):494,(5,90):534,(5,91):1091,(5,92):2027,(5,93):1410,(5,94):1797,(5,95):370,(5,96):560,(5,97):656,(5,98):2059,(5,99):2062,(5,100):320,(5,101):1138,(5,102):1460,(5,103):915,(5,104):144,(5,105):572,(5,106):1693,(5,107):941,(5,108):982,(5,109):1403,(5,110):98,(5,111):417,(5,112):913,(5,113):2103,(5,114):781,(5,115):1997,(5,116):33,(5,117):468,(5,119):974,(5,121):842,(5,118):828,(5,120):800,(5,122):821,(5,123):1790,(5,124):2175,(5,125):2030,(5,126):816,(5,127):513,(5,128):2016,(5,129):2038,(5,130):1166,(5,131):500,(5,132):2107,(5,133):2025,(5,134):1796,(5,135):1112,(5,136):2044,(5,137):825,(5,138):1382,(5,140):783,(5,141):1708,(5,142):483,(5,143):929,(5,144):2025,(5,146):1284,(5,145):2067,(5,147):981,(5,148):752,(5,150):1985,(5,151):727,(5,152):2008,(5,153):609,(5,154):92,(5,155):2108,(5,156):310,(5,157):409,(5,158):1197,(5,159):2092,(5,160):2171,(5,161):1484,(5,139):1192,(5,162):1240,(5,163):1994,(5,164):2037,(5,165):2189,(5,166):2166,(5,167):2044,(5,168):2057,(5,169):2186,(5,170):907,(5,171):1690,(5,172):2044,(5,173):864,(5,174):561,(5,175):1773,(5,178):1802,(5,176):521,(5,177):912,(5,179):2114,(5,180):2174,(5,181):1788,(5,182):696,(5,183):2050,(5,184):956,(5,185):1184,(5,186):1706,(5,187):246,(5,188):842,(5,189):2068,(5,190):1698,(5,191):703,(5,192):2066,(5,193):840,(5,194):2041,(5,195):281,(5,196):700,(5,197):696,(5,198):662,(5,199):955,(5,200):842,(6,1):245,(6,2):1469,(6,3):469,(6,4):1166,(6,5):2215,(6,7):982,(6,8):468,(6,9):435,(6,10):488,(6,11):1271,(6,12):319,(6,13):1138,(6,14):2245,(6,15):498,(6,16):832,(6,17):2310,(6,18):478,(6,19):1962,(6,20):871,(6,21):723,(6,6):1262,(6,22):1392,(6,23):463,(6,24):1029,(6,25):955,(6,26):457,(6,27):501,(6,28):1852,(6,29):397,(6,30):332,(6,31):562,(6,32):297,(6,33):2199,(6,34):2,(6,35):260,(6,36):1280,(6,37):475,(6,38):119,(6,39):538,(6,77):861,(6,40):2195,(6,41):1269,(6,42):957,(6,43):58,(6,149):1356,(6,44):958,(6,45):1280,(6,46):600,(6,47):280,(6,48):441,(6,49):2228,(6,51):1566,(6,50):2327,(6,52):2185,(6,53):2365,(6,54):490,(6,55):2181,(6,56):1289,(6,57):1097,(6,58):164,(6,59):991,(6,60):2383,(6,61):2261,(6,62):2217,(6,63):2222,(6,64):944,(6,65):1844,(6,66):1864,(6,67):2229,(6,68):976,(6,69):321,(6,70):389,(6,71):552,(6,72):2387,(6,73):1976,(6,74):1116,(6,75):1287,(6,76):1105,(6,78):1053,(6,79):2228,(6,80):384,(6,81):117,(6,82):2215,(6,83):965,(6,84):669,(6,85):739,(6,86):664,(6,87):299,(6,88):635,(6,89):646,(6,90):258,(6,91):1295,(6,92):2210,(6,93):1386,(6,94):1985,(6,95):86,(6,96):773,(6,97):623,(6,98):2234,(6,99):2241,(6,100):106,(6,101):1223,(6,102):1407,(6,103):931,(6,104):441,(6,105):483,(6,106):1840,(6,107):949,(6,108):833,(6,109):1124,(6,110):386,(6,111):712,(6,112):727,(6,113):2317,(6,114):562,(6,115):2170,(6,116):312,(6,117):280,(6,119):820,(6,121):671,(6,118):658,(6,120):538,(6,122):559,(6,123):1979,(6,124):2392,(6,125):2197,(6,126):892,(6,127):736,(6,128):2192,(6,129):2212,(6,130):889,(6,131):644,(6,132):2289,(6,133):2207,(6,134):1983,(6,135):1049,(6,136):2222,(6,137):661,(6,138):1104,(6,140):592,(6,141):1857,(6,142):303,(6,143):944,(6,144):2201,(6,146):1003,(6,145):2334,(6,147):839,(6,148):462,(6,150):2211,(6,151):467,(6,152):2182,(6,153):536,(6,154):381,(6,155):2330,(6,156):366,(6,157):704,(6,158):932,(6,159):2356,(6,160):2379,(6,161):1711,(6,139):1229,(6,162):1225,(6,163):2169,(6,164):2200,(6,165):2406,(6,166):2378,(6,167):2218,(6,168):2239,(6,169):2408,(6,170):618,(6,171):1840,(6,172):2320,(6,173):817,(6,174):829,(6,175):2051,(6,178):1989,(6,176):583,(6,177):778,(6,179):2331,(6,180):2388,(6,181):1975,(6,182):604,(6,183):2325,(6,184):704,(6,185):918,(6,186):1853,(6,187):215,(6,188):652,(6,189):2245,(6,190):1827,(6,191):775,(6,192):2333,(6,193):576,(6,194):2241,(6,195):290,(6,196):473,(6,197):827,(6,198):367,(6,199):824,(6,200):679,(7,1):127,(7,2):1571,(7,3):384,(7,4):1273,(7,5):2310,(7,7):1099,(7,8):382,(7,9):516,(7,10):536,(7,12):356,(7,11):1355,(7,13):1256,(7,14):2335,(7,15):404,(7,16):946,(7,17):2357,(7,18):581,(7,19):2025,(7,20):758,(7,21):613,(7,6):1373,(7,22):1509,(7,23):345,(7,24):1089,(7,25):1072,(7,26):436,(7,27):556,(7,28):1954,(7,29):414,(7,30):426,(7,31):505,(7,32):325,(7,33):2298,(7,34):117,(7,35):146,(7,36):1369,(7,37):503,(7,38):0,(7,39):621,(7,77):745,(7,40):2290,(7,41):1387,(7,42):1074,(7,43):76,(7,149):1474,(7,44):1074,(7,45):1363,(7,46):667,(7,47):194,(7,48):419,(7,49):2323,(7,51):1676,(7,50):2407,(7,52):2283,(7,53):2425,(7,54):480,(7,55):2276,(7,56):1368,(7,57):1145,(7,58):158,(7,59):1108,(7,60):2465,(7,61):2347,(7,62):2312,(7,63):2317,(7,64):1061,(7,65):1946,(7,66):1965,(7,67):2323,(7,68):1093,(7,69):291,(7,70):378,(7,71):490,(7,72):2469,(7,73):2067,(7,74):1165,(7,75):1372,(7,76):1153,(7,78):1172,(7,79):2323,(7,80):490,(7,81):197,(7,82):2311,(7,83):1082,(7,84):783,(7,85):792,(7,86):557,(7,87):339,(7,88):728,(7,89):739,(7,90):333,(7,91):1379,(7,92):2303,(7,93):1504,(7,94):2077,(7,95):183,(7,96):851,(7,97):741,(7,98):2330,(7,99):2335,(7,100):225,(7,101):1334,(7,102):1525,(7,103):1048,(7,104):463,(7,105):602,(7,106):1942,(7,107):1066,(7,108):943,(7,109):1173,(7,110):392,(7,111):737,(7,112):830,(7,113):2400,(7,114):658,(7,115):2266,(7,116):348,(7,117):394,(7,119):930,(7,121):565,(7,118):551,(7,120):476,(7,122):498,(7,123):2070,(7,124):2473,(7,125):2295,(7,126):1003,(7,127):810,(7,128):2287,(7,129):2308,(7,130):942,(7,131):739,(7,132):2382,(7,133):2300,(7,134):2074,(7,135):1167,(7,136):2317,(7,137):552,(7,138):1153,(7,140):491,(7,141):1958,(7,142):191,(7,143):1061,(7,144):2296,(7,146):1050,(7,145):2389,(7,147):727,(7,148):439,(7,150):2289,(7,151):404,(7,152):2277,(7,153):417,(7,154):415,(7,155):2409,(7,156):469,(7,157):728,(7,158):996,(7,159):2413,(7,160):2464,(7,161):1788,(7,139):1345,(7,162):1343,(7,163):2264,(7,164):2298,(7,165):2488,(7,166):2462,(7,167):2314,(7,168):2332,(7,169):2488,(7,170):661,(7,171):1941,(7,172):2367,(7,173):936,(7,174):881,(7,175):2097,(7,178):2080,(7,176):693,(7,177):665,(7,179):2413,(7,180):2471,(7,181):2066,(7,182):486,(7,183):2373,(7,184):781,(7,185):980,(7,186):1955,(7,187):140,(7,188):551,(7,189):2340,(7,190):1933,(7,191):885,(7,192):2388,(7,193):517,(7,194):2329,(7,195):203,(7,196):386,(7,197):927,(7,198):362,(7,199):710,(7,200):571,(8,1):1199,(8,2):689,(8,3):1388,(8,4):391,(8,5):1434,(8,7):24,(8,8):1389,(8,9):846,(8,10):1011,(8,12):915,(8,11):767,(8,13):212,(8,14):1509,(8,15):1427,(8,16):436,(8,17):1971,(8,18):684,(8,19):1518,(8,20):1828,(8,21):1674,(8,6):375,(8,22):560,(8,23):1412,(8,24):1159,(8,25):17,(8,26):1234,(8,27):818,(8,28):1029,(8,29):1095,(8,30):801,(8,31):1405,(8,32):943,(8,33):1385,(8,34):960,(8,35):1207,(8,36):721,(8,37):1085,(8,38):1077,(8,39):818,(8,77):1820,(8,40):1414,(8,41):415,(8,42):2,(8,43):1003,(8,149):399,(8,44):41,(8,45):776,(8,46):745,(8,47):1177,(8,48):1228,(8,49):1454,(8,51):667,(8,50):1685,(8,52):1383,(8,53):1901,(8,54):1222,(8,55):1406,(8,56):831,(8,57):1297,(8,58):1014,(8,59):37,(8,60):1716,(8,61):1568,(8,62):1438,(8,63):1440,(8,64):15,(8,65):1021,(8,66):1051,(8,67):1459,(8,68):17,(8,69):1093,(8,70):1169,(8,71):1410,(8,72):1724,(8,73):1249,(8,74):1298,(8,75):768,(8,76):1300,(8,78):264,(8,79):1444,(8,80):706,(8,81):900,(8,82):1430,(8,83):13,(8,84):454,(8,85):1068,(8,86):1612,(8,87):907,(8,88):536,(8,89):533,(8,90):901,(8,91):783,(8,92):1454,(8,93):468,(8,94):1265,(8,95):923,(8,96):654,(8,97):339,(8,98):1454,(8,99):1470,(8,100):855,(8,101):352,(8,102):544,(8,103):35,(8,104):963,(8,105):496,(8,106):1020,(8,107):13,(8,108):513,(8,109):1307,(8,110):1009,(8,111):1016,(8,112):627,(8,113):1653,(8,114):722,(8,115):1389,(8,116):920,(8,117):726,(8,119):526,(8,121):1618,(8,118):1606,(8,120):1396,(8,122):1412,(8,123):1261,(8,124):1732,(8,125):1399,(8,126):223,(8,127):692,(8,128):1416,(8,129):1431,(8,130):1135,(8,131):518,(8,132):1523,(8,133):1448,(8,134):1258,(8,135):279,(8,136):1451,(8,137):1611,(8,138):1290,(8,140):1532,(8,141):1042,(8,142):1261,(8,143):21,(8,144):1426,(8,146):1242,(8,145):1922,(8,147):1793,(8,148):1243,(8,150):1605,(8,151):1341,(8,152):1402,(8,153):1488,(8,154):919,(8,155):1693,(8,156):644,(8,157):1018,(8,158):1076,(8,159):1922,(8,160):1688,(8,161):1180,(8,139):275,(8,162):296,(8,163):1394,(8,164):1390,(8,165):1747,(8,166):1705,(8,167):1435,(8,168):1476,(8,169):1769,(8,170):1078,(8,171):1028,(8,172):1981,(8,173):207,(8,174):876,(8,175):1754,(8,178):1266,(8,176):428,(8,177):1735,(8,179):1676,(8,180):1717,(8,181):1254,(8,182):1562,(8,183):1972,(8,184):888,(8,185):1079,(8,186):1033,(8,187):1119,(8,188):1589,(8,189):1466,(8,190):968,(8,191):278,(8,192):1924,(8,193):1423,(8,194):1534,(8,195):1186,(8,196):1394,(8,197):401,(8,198):1142,(8,199):1780,(8,200):1629,(9,1):1455,(9,2):391,(9,3):1754,(9,4):423,(9,5):966,(9,7):772,(9,8):1752,(9,9):1425,(9,10):1584,(9,11):17,(9,12):1042,(9,13):907,(9,14):971,(9,15):1776,(9,16):1192,(9,17):1192,(9,18):1289,(9,19):751,(9,20):2084,(9,21):1963,(9,6):564,(9,22):1236,(9,23):1612,(9,24):1887,(9,25):764,(9,26):1711,(9,27):839,(9,28):691,(9,29):1600,(9,30):1329,(9,31):1845,(9,32):1079,(9,33):980,(9,34):1287,(9,35):1444,(9,36):74,(9,37):1632,(9,38):1372,(9,39):1445,(9,77):2047,(9,40):948,(9,41):1110,(9,42):782,(9,43):1299,(9,149):889,(9,44):740,(9,45):8,(9,46):718,(9,47):1358,(9,48):1699,(9,49):975,(9,51):650,(9,50):1039,(9,52):955,(9,53):1150,(9,54):1724,(9,55):931,(9,56):66,(9,57):2018,(9,58):1233,(9,59):759,(9,60):1095,(9,61):975,(9,62):968,(9,63):974,(9,64):781,(9,65):684,(9,66):688,(9,67):973,(9,68):777,(9,69):1208,(9,70):1639,(9,71):1838,(9,72):1099,(9,73):710,(9,74):2023,(9,75):16,(9,76):2022,(9,78):1035,(9,79):981,(9,80):1257,(9,81):1178,(9,82):971,(9,83):769,(9,84):1144,(9,85):1728,(9,86):1915,(9,87):1051,(9,88):658,(9,89):647,(9,90):1384,(9,91):8,(9,92):948,(9,93):1062,(9,94):716,(9,95):1306,(9,96):523,(9,97):915,(9,98):985,(9,99):985,(9,100):1223,(9,101):542,(9,102):1194,(9,103):763,(9,104):989,(9,105):1039,(9,106):678,(9,107):791,(9,108):1266,(9,109):2033,(9,110):1072,(9,111):822,(9,112):1342,(9,113):1029,(9,114):1368,(9,115):925,(9,116):1050,(9,117):1203,(9,119):1275,(9,121):1923,(9,118):1909,(9,120):1823,(9,122):1844,(9,123):709,(9,124):1104,(9,125):964,(9,126):595,(9,127):569,(9,128):942,(9,129):965,(9,130):1829,(9,131):653,(9,132):1028,(9,133):946,(9,134):716,(9,135):1052,(9,136):967,(9,137):1907,(9,138):2013,(9,140):1858,(9,141):684,(9,142):1554,(9,143):767,(9,144):950,(9,146):1948,(9,145):1155,(9,147):2064,(9,148):1719,(9,150):929,(9,151):1754,(9,152):935,(9,153):1686,(9,154):999,(9,155):1043,(9,156):935,(9,157):830,(9,158):1794,(9,159):1161,(9,160):1092,(9,161):439,(9,139):740,(9,162):943,(9,163):920,(9,164):976,(9,165):1118,(9,166):1090,(9,167):971,(9,168):978,(9,169):1122,(9,170):1690,(9,171):666,(9,172):1202,(9,173):944,(9,174):589,(9,175):972,(9,178):721,(9,176):774,(9,177):1994,(9,179):1043,(9,180):1100,(9,181):707,(9,182):1774,(9,183):1195,(9,184):1569,(9,185):1793,(9,186):688,(9,187):1328,(9,188):1918,(9,189):993,(9,190):727,(9,191):653,(9,192):1156,(9,193):1860,(9,194):959,(9,195):1362,(9,196):1757,(9,197):514,(9,198):1611,(9,199):2037,(9,200):1924,(10,1):303,(10,2):1376,(10,3):573,(10,4):1080,(10,5):2115,(10,7):924,(10,8):572,(10,9):502,(10,10):580,(10,12):211,(10,11):1164,(10,13):1092,(10,14):2141,(10,15):597,(10,16):829,(10,17):2195,(10,18):510,(10,19):1848,(10,20):948,(10,21):807,(10,6):1184,(10,22):1367,(10,23):510,(10,24):1105,(10,25):896,(10,26):571,(10,27):386,(10,28):1759,(10,29):505,(10,30):387,(10,31):675,(10,32):194,(10,33):2103,(10,34):113,(10,35):307,(10,36):1175,(10,37):577,(10,38):195,(10,39):597,(10,77):930,(10,40):2095,(10,41):1238,(10,42):901,(10,43):119,(10,149):1302,(10,44):897,(10,45):1172,(10,46):486,(10,47):281,(10,48):556,(10,49):2128,(10,51):1485,(10,50):2218,(10,52):2088,(10,53):2250,(10,54):603,(10,55):2081,(10,56):1179,(10,57):1182,(10,58):121,(10,59):932,(10,60):2275,(10,61):2155,(10,62):2118,(10,63):2123,(10,64):887,(10,65):1751,(10,66):1770,(10,67):2128,(10,68):919,(10,69):261,(10,70):503,(10,71):664,(10,72):2279,(10,73):1873,(10,74):1200,(10,75):1180,(10,76):1190,(10,78):1023,(10,79):2129,(10,80):413,(10,81):2,(10,82):2116,(10,83):907,(10,84):664,(10,85):824,(10,86):752,(10,87):189,(10,88):533,(10,89):544,(10,90):339,(10,91):1188,(10,92):2109,(10,93):1345,(10,94):1883,(10,95):175,(10,96):662,(10,97):577,(10,98):2135,(10,99):2141,(10,100):128,(10,101):1145,(10,102):1377,(10,103):872,(10,104):335,(10,105):456,(10,106):1747,(10,107):893,(10,108):841,(10,109):1208,(10,110):290,(10,111):603,(10,112):752,(10,113):2210,(10,114):604,(10,115):2071,(10,116):205,(10,117):296,(10,119):830,(10,121):759,(10,118):745,(10,120):650,(10,122):671,(10,123):1876,(10,124):2284,(10,125):2100,(10,126):814,(10,127):624,(10,128):2092,(10,129):2113,(10,130):972,(10,131):544,(10,132):2188,(10,133):2106,(10,134):1880,(10,135):1021,(10,136):2122,(10,137):747,(10,138):1188,(10,140):686,(10,141):1764,(10,142):385,(10,143):885,(10,144):2101,(10,146):1089,(10,145):2219,(10,147):920,(10,148):577,(10,150):2101,(10,151):579,(10,152):2083,(10,153):587,(10,154):272,(10,155):2221,(10,156):276,(10,157):595,(10,158):1006,(10,159):2241,(10,160):2273,(10,161):1601,(10,139):1165,(10,162):1179,(10,163):2070,(10,164):2103,(10,165):2298,(10,166):2271,(10,167):2119,(10,168):2138,(10,169):2299,(10,170):711,(10,171):1746,(10,172):2205,(10,173):780,(10,174):714,(10,175):1936,(10,178):1886,(10,176):503,(10,177):855,(10,179):2223,(10,180):2280,(10,181):1872,(10,182):664,(10,183):2210,(10,184):768,(10,185):993,(10,186):1760,(10,187):219,(10,188):746,(10,189):2145,(10,190):1739,(10,191):696,(10,192):2218,(10,193):689,(10,194):2136,(10,195):290,(10,196):576,(10,197):733,(10,198):480,(10,199):900,(10,200):765,(11,1):876,(11,2):1750,(11,3):756,(11,4):1434,(11,5):2503,(11,7):1091,(11,8):760,(11,9):338,(11,10):256,(11,11):1712,(11,12):1028,(11,13):1131,(11,14):2574,(11,15):804,(11,16):666,(11,17):2882,(11,18):441,(11,19):2473,(11,20):1199,(11,21):1045,(11,6):1445,(11,22):1191,(11,23):1039,(11,24):301,(11,25):1078,(11,26):484,(11,27):1153,(11,28):2098,(11,29):400,(11,30):458,(11,31):631,(11,32):1017,(11,33):2455,(11,34):742,(11,35):908,(11,36):1692,(11,37):306,(11,38):792,(11,39):285,(11,77):1239,(11,40):2483,(11,41):1133,(11,42):1067,(11,43):788,(11,149):1361,(11,44):1096,(11,45):1722,(11,46):1207,(11,47):986,(11,48):496,(11,49):2523,(11,51):1730,(11,50):2728,(11,52):2453,(11,53):2877,(11,54):429,(11,55):2475,(11,56):1756,(11,57):357,(11,58):904,(11,59):1106,(11,60):2768,(11,61):2624,(11,62):2507,(11,63):2510,(11,64):1059,(11,65):2091,(11,66):2120,(11,67):2527,(11,68):1083,(11,69):1061,(11,70):484,(11,71):656,(11,72):2774,(11,73):2309,(11,74):376,(11,75):1722,(11,76):366,(11,78):969,(11,79):2514,(11,80):494,(11,81):826,(11,82):2500,(11,83):1082,(11,84):617,(11,85):0,(11,86):983,(11,87):1006,(11,88):1120,(11,89):1129,(11,90):485,(11,91):1734,(11,92):2521,(11,93):1291,(11,94):2324,(11,95):656,(11,96):1309,(11,97):817,(11,98):2523,(11,99):2538,(11,100):699,(11,101):1421,(11,102):1235,(11,103):1064,(11,104):1154,(11,105):692,(11,106):2089,(11,107):1056,(11,108):601,(11,109):384,(11,110):1115,(11,111):1405,(11,112):453,(11,113):2703,(11,114):361,(11,115):2458,(11,116):1024,(11,117):590,(11,119):583,(11,121):984,(11,118):980,(11,120):650,(11,122):650,(11,123):2319,(11,124):2782,(11,125):2469,(11,126):1161,(11,127):1298,(11,128):2485,(11,129):2500,(11,130):150,(11,131):1117,(11,132):2592,(11,133):2515,(11,134):2318,(11,135):953,(11,136):2520,(11,137):993,(11,138):364,(11,140):899,(11,141):2112,(11,142):831,(11,143):1069,(11,144):2495,(11,146):265,(11,145):2872,(11,147):1151,(11,148):491,(11,150):2635,(11,151):651,(11,152):2471,(11,153):1078,(11,154):1086,(11,155):2735,(11,156):888,(11,157):1398,(11,158):220,(11,159):2883,(11,160):2745,(11,161):2166,(11,139):1317,(11,162):1192,(11,163):2463,(11,164):2460,(11,165):2797,(11,166):2758,(11,167):2505,(11,168):2544,(11,169):2813,(11,170):142,(11,171):2098,(11,172):2892,(11,173):862,(11,174):1452,(11,175):2634,(11,178):2325,(11,176):965,(11,177):1130,(11,179):2724,(11,180):2770,(11,181):2313,(11,182):1095,(11,183):2891,(11,184):185,(11,185):201,(11,186):2102,(11,187):931,(11,188):927,(11,189):2535,(11,190):2035,(11,191):1079,(11,192):2872,(11,193):646,(11,194):2593,(11,195):994,(11,196):764,(11,197):1215,(11,198):479,(11,199):1166,(11,200):1006,(12,1):830,(12,2):844,(12,3):1107,(12,4):566,(12,5):1578,(12,8):1106,(12,7):548,(12,9):796,(12,10):946,(12,12):447,(12,11):639,(12,13):746,(12,14):1606,(12,15):1134,(12,16):742,(12,17):1761,(12,18):683,(12,19):1368,(12,20):1474,(12,21):1340,(12,6):697,(12,22):1091,(12,23):1014,(12,24):1329,(12,25):520,(12,26):1055,(12,27):295,(12,28):1227,(12,29):947,(12,30):689,(12,31):1190,(12,32):483,(12,33):1567,(12,34):637,(12,35):825,(12,36):643,(12,37):986,(12,38):732,(12,39):836,(12,77):1448,(12,40):1559,(12,41):946,(12,42):533,(12,43):656,(12,149):899,(12,44):508,(12,45):648,(12,46):210,(12,47):757,(12,48):1043,(12,49):1591,(12,51):981,(12,50):1695,(12,52):1551,(12,53):1774,(12,54):1070,(12,55):1545,(12,56):665,(12,57):1445,(12,58):610,(12,59):548,(12,60):1749,(12,61):1624,(12,62):1581,(12,63):1586,(12,64):522,(12,65):1219,(12,66):1236,(12,67):1592,(12,68):546,(12,69):632,(12,70):983,(12,71):1184,(12,72):1754,(12,73):1337,(12,74):1455,(12,75):654,(12,76):1451,(12,78):762,(12,79):1592,(12,80):630,(12,81):534,(12,82):1580,(12,83):532,(12,84):624,(12,85):1121,(12,86):1287,(12,87):448,(12,88):3,(12,89):8,(12,90):733,(12,91):663,(12,92):1573,(12,93):999,(12,94):1347,(12,95):651,(12,96):191,(12,97):386,(12,98):1598,(12,99):1604,(12,100):568,(12,101):655,(12,102):1078,(12,103):499,(12,104):455,(12,105):441,(12,106):1214,(12,107):532,(12,108):798,(12,109):1464,(12,110):519,(12,111):483,(12,112):812,(12,113):1683,(12,114):777,(12,115):1534,(12,116):454,(12,117):558,(12,119):799,(12,121):1294,(12,118):1280,(12,120):1169,(12,122):1189,(12,123):1341,(12,124):1759,(12,125):1563,(12,126):350,(12,127):196,(12,128):1555,(12,129):1576,(12,130):1241,(12,131):16,(12,132):1651,(12,133):1569,(12,134):1344,(12,135):770,(12,136):1585,(12,137):1281,(12,138):1444,(12,140):1222,(12,141):1230,(12,142):920,(12,143):513,(12,144):1565,(12,146):1364,(12,145):1759,(12,147):1449,(12,148):1063,(12,150):1584,(12,151):1100,(12,152):1546,(12,153):1091,(12,154):428,(12,155):1699,(12,156):279,(12,157):484,(12,158):1228,(12,159):1774,(12,160):1743,(12,161):1089,(12,139):735,(12,162):828,(12,163):1533,(12,164):1567,(12,165):1773,(12,166):1744,(12,167):1582,(12,168):1601,(12,169):1778,(12,170):1061,(12,171):1212,(12,172):1771,(12,173):543,(12,174):384,(12,175):1512,(12,178):1351,(12,176):176,(12,177):1382,(12,179):1698,(12,180):1753,(12,181):1337,(12,182):1175,(12,183):1771,(12,184):985,(12,185):1223,(12,186):1227,(12,187):714,(12,188):1283,(12,189):1608,(12,190):1213,(12,191):256,(12,192):1759,(12,193):1206,(12,194):1604,(12,195):764,(12,196):1111,(12,197):209,(12,198):956,(12,199):1426,(12,200):1298,(13,1):2161,(13,2):569,(13,3):2450,(13,4):883,(13,5):264,(13,7):1239,(13,8):2448,(13,9):2050,(13,10):2216,(13,11):726,(13,12):1751,(13,13):1274,(13,14):264,(13,15):2475,(13,16):1693,(13,17):1023,(13,18):1898,(13,19):609,(13,20):2795,(13,21):2671,(13,6):894,(13,22):1483,(13,23):2324,(13,24):2416,(13,25):1247,(13,26):2380,(13,27):1550,(13,28):319,(13,29):2258,(13,30):1971,(13,31):2526,(13,32):1789,(13,33):316,(13,34):1979,(13,35):2152,(13,36):703,(13,37):2276,(13,38):2072,(13,39):2048,(13,77):2759,(13,40):248,(13,41):1409,(13,42):1260,(13,43):1997,(13,149):1119,(13,44):1227,(13,45):720,(13,46):1428,(13,47):2068,(13,48):2370,(13,49):267,(13,51):683,(13,50):443,(13,52):279,(13,53):811,(13,54):2386,(13,55):229,(13,56):734,(13,57):2556,(13,58):1939,(13,59):1224,(13,60):461,(13,61):309,(13,62):264,(13,63):271,(13,64):1267,(13,65):322,(13,66):287,(13,67):264,(13,68):1247,(13,69):1920,(13,70):2309,(13,71):2522,(13,72):469,(13,73):14,(13,74):2556,(13,75):708,(13,76):2558,(13,78):1442,(13,79):279,(13,80):1888,(13,81):1875,(13,82):271,(13,83):1246,(13,84):1701,(13,85):2317,(13,86):2622,(13,87):1760,(13,88):1345,(13,89):1334,(13,90):2043,(13,91):704,(13,92):236,(13,93):1288,(13,94):7,(13,95):1989,(13,96):1231,(13,97):1512,(13,98):280,(13,99):276,(13,100):1906,(13,101):909,(13,102):1425,(13,103):1258,(13,104):1701,(13,105):1660,(13,106):318,(13,107):1272,(13,108):1769,(13,109):2565,(13,110):1784,(13,111):1523,(13,112):1885,(13,113):400,(13,114):1959,(13,115):229,(13,116):1760,(13,117):1857,(13,119):1783,(13,121):2629,(13,118):2615,(13,120):2508,(13,122):2527,(13,123):8,(13,124):479,(13,125):281,(13,126):1163,(13,127):1280,(13,128):239,(13,129):263,(13,130):2392,(13,131):1336,(13,132):316,(13,133):234,(13,134):7,(13,135):1461,(13,136):259,(13,137):2614,(13,138):2548,(13,140):2561,(13,141):295,(13,142):2258,(13,143):1255,(13,144):246,(13,146):2501,(13,145):892,(13,147):2774,(13,148):2389,(13,150):408,(13,151):2440,(13,152):237,(13,153):2398,(13,154):1710,(13,155):454,(13,156):1617,(13,157):1532,(13,158):2334,(13,159):863,(13,160):429,(13,161):426,(13,139):1063,(13,162):1259,(13,163):219,(13,164):305,(13,165):494,(13,166):448,(13,167):270,(13,168):267,(13,169):524,(13,170):2306,(13,171):296,(13,172):1029,(13,173):1462,(13,174):1293,(13,175):946,(13,178):9,(13,176):1425,(13,177):2705,(13,179):428,(13,180):461,(13,181):4,(13,182):2486,(13,183):1005,(13,184):2141,(13,185):2338,(13,186):311,(13,187):2036,(13,188):2621,(13,189):287,(13,190):431,(13,191):1261,(13,192):897,(13,193):2543,(13,194):276,(13,195):2073,(13,196):2453,(13,197):1162,(13,198):2281,(13,199):2748,(13,200):2631,(14,1):2423,(14,2):785,(14,3):2704,(14,4):1105,(14,5):27,(14,7):1438,(14,8):2703,(14,9):2278,(14,10):2444,(14,11):996,(14,12):2016,(14,13):1442,(14,14):123,(14,15):2732,(14,16):1885,(14,17):1135,(14,18):2122,(14,19):795,(14,20):3060,(14,21):2933,(14,6):1085,(14,22):1601,(14,23):2590,(14,24):2604,(14,25):1450,(14,26):2623,(14,27):1817,(14,28):436,(14,29):2496,(14,30):2206,(14,31):2774,(14,32):2054,(14,33):136,(14,34):2234,(14,35):2415,(14,36):968,(14,37):2508,(14,38):2330,(14,39):2267,(14,77):3026,(14,40):46,(14,41):1548,(14,42):1461,(14,43):2254,(14,149):1258,(14,44):1432,(14,45):990,(14,46):1695,(14,47):2334,(14,48):2613,(14,49):7,(14,51):825,(14,50):414,(14,52):104,(14,53):876,(14,54):2625,(14,55):54,(14,56):1007,(14,57):2745,(14,58):2201,(14,59):1423,(14,60):381,(14,61):250,(14,62):22,(14,63):25,(14,64):1469,(14,65):443,(14,66):410,(14,67):16,(14,68):1446,(14,69):2187,(14,70):2552,(14,71):2771,(14,72):391,(14,73):273,(14,74):2743,(14,75):977,(14,76):2747,(14,78):1614,(14,79):28,(14,80):2119,(14,81):2132,(14,82):39,(14,83):1447,(14,84):1911,(14,85):2527,(14,86):2883,(14,87):2024,(14,88):1601,(14,89):1590,(14,90):2283,(14,91):974,(14,92):58,(14,93):1411,(14,94):271,(14,95):2238,(14,96):1497,(14,97):1738,(14,98):16,(14,99):15,(14,100):2156,(14,101):1107,(14,102):1541,(14,103):1464,(14,104):1971,(14,105):1891,(14,106):443,(14,107):1473,(14,108):1960,(14,109):2752,(14,110):2053,(14,111):1797,(14,112):2086,(14,113):350,(14,114):2175,(14,115):70,(14,116):2025,(14,117):2096,(14,119):1975,(14,121):2890,(14,118):2876,(14,120):2756,(14,122):2776,(14,123):279,(14,124):405,(14,125):87,(14,126):1391,(14,127):1547,(14,128):43,(14,129):31,(14,130):2593,(14,131):1591,(14,132):67,(14,133):52,(14,134):269,(14,135):1633,(14,136):16,(14,137):2876,(14,138):2736,(14,140):2820,(14,141):419,(14,142):2518,(14,143):1459,(14,144):33,(14,146):2696,(14,145):981,(14,147):3039,(14,148):2631,(14,150):464,(14,151):2690,(14,152):57,(14,153):2665,(14,154):1977,(14,155):430,(14,156):1870,(14,157):1806,(14,158):2528,(14,159):940,(14,160):315,(14,161):688,(14,139):1234,(14,162):1414,(14,163):66,(14,164):121,(14,165):415,(14,166):360,(14,167):31,(14,168):39,(14,169):467,(14,170):2527,(14,171):432,(14,172):1139,(14,173):1666,(14,174):1566,(14,175):1117,(14,178):265,(14,176):1669,(14,177):2970,(14,179):382,(14,180):374,(14,181):279,(14,182):2753,(14,183):1112,(14,184):2348,(14,185):2533,(14,186):430,(14,187):2300,(14,188):2880,(14,189):13,(14,190):522,(14,191):1496,(14,192):988,(14,193):2791,(14,194):204,(14,195):2339,(14,196):2708,(14,197):1410,(14,198):2524,(14,199):3013,(14,200):2893,(26,1):1446,(26,2):344,(26,3):1676,(26,4):131,(26,5):1085,(26,7):329,(26,8):1676,(26,9):1180,(26,10):1346,(26,11):533,(26,12):1097,(26,13):389,(26,14):1159,(26,15):1712,(26,16):782,(26,17):1688,(26,18):1020,(26,19):1223,(26,20):2090,(26,21):1944,(26,6):43,(26,22):695,(26,23):1647,(26,24):1505,(26,25):340,(26,26):1550,(26,27):945,(26,28):681,(26,29):1416,(26,30):1121,(26,31):1714,(26,32):1131,(26,33):1038,(26,34):1221,(26,35):1448,(26,36):470,(26,37):1416,(26,38):1333,(26,39):1161,(26,77):2073,(26,40):1064,(26,41):575,(26,42):352,(26,43):1255,(26,149):350,(26,44):322,(26,45):541,(26,46):839,(26,47):1396,(26,48):1543,(26,49):1104,(26,51):345,(26,50):1342,(26,52):1035,(26,53):1589,(26,54):1545,(26,55):1056,(26,56):605,(26,57):1645,(26,58):1239,(26,59):314,(26,60):1370,(26,61):1220,(26,62):1088,(26,63):1091,(26,64):359,(26,65):673,(26,66):702,(26,67):1109,(26,68):337,(26,69):1282,(26,70):1483,(26,71):1716,(26,72):1378,(26,73):901,(26,74):1645,(26,75):527,(26,76):1647,(26,78):544,(26,79):1095,(26,80):1029,(26,81):1140,(26,82):1081,(26,83):337,(26,84):801,(26,85):1417,(26,86):1886,(26,87):1095,(26,88):657,(26,89):648,(26,90):1211,(26,91):541,(26,92):1103,(26,93):520,(26,94):918,(26,95):1202,(26,96):677,(26,97):646,(26,98):1104,(26,99):1120,(26,100):1126,(26,101):2,(26,102):651,(26,103):354,(26,104):1107,(26,105):805,(26,106):672,(26,107):363,(26,108):858,(26,109):1654,(26,110):1173,(26,111):1071,(26,112):977,(26,113):1307,(26,114):1067,(26,115):1040,(26,116):1103,(26,117):1027,(26,119):872,(26,121):1893,(26,118):1879,(26,120):1702,(26,122):1719,(26,123):913,(26,124):1387,(26,125):1051,(26,126):329,(26,127):730,(26,128):1066,(26,129):1081,(26,130):1485,(26,131):639,(26,132):1173,(26,133):1097,(26,134):910,(26,135):563,(26,136):1102,(26,137):1882,(26,138):1636,(26,140):1812,(26,141):693,(26,142):1523,(26,143):349,(26,144):1076,(26,146):1591,(26,145):1622,(26,147):2060,(26,148):1560,(26,150):1269,(26,151):1642,(26,152):1052,(26,153):1724,(26,154):1082,(26,155):1351,(26,156):866,(26,157):1076,(26,158):1424,(26,159):1616,(26,160):1340,(26,161):877,(26,139):198,(26,162):408,(26,163):1044,(26,164):1043,(26,165):1402,(26,166):1359,(26,167):1086,(26,168):1126,(26,169):1426,(26,170):1422,(26,171):679,(26,172):1697,(26,173):556,(26,174):873,(26,175):1491,(26,178):918,(26,176):639,(26,177):1997,(26,179):1332,(26,180):1371,(26,181):906,(26,182):1805,(26,183):1685,(26,184):1238,(26,185):1428,(26,186):685,(26,187):1345,(26,188):1870,(26,189):1116,(26,190):628,(26,191):447,(26,192):1624,(26,193):1732,(26,194):1186,(26,195):1403,(26,196):1682,(26,197):448,(26,198):1454,(26,199):2043,(26,200):1901,(15,1):730,(15,2):1111,(15,3):890,(15,4):791,(15,5):1873,(15,7):522,(15,8):891,(15,9):384,(15,10):550,(15,12):558,(15,11):1029,(15,13):659,(15,14):1928,(15,15):931,(15,16):382,(15,17):2199,(15,18):243,(15,19):1788,(15,20):1338,(15,21):1181,(15,6):845,(15,22):910,(15,23):948,(15,24):880,(15,25):498,(15,26):741,(15,27):575,(15,28):1482,(15,29):606,(15,30):311,(15,31):908,(15,32):572,(15,33):1839,(15,34):485,(15,35):744,(15,36):1013,(15,37):612,(15,38):603,(15,39):399,(15,77):1336,(15,40):1853,(15,41):785,(15,42):496,(15,43):535,(15,149):885,(15,44):507,(15,45):1039,(15,46):582,(15,47):739,(15,48):734,(15,49):1890,(15,51):1150,(15,50):2061,(15,52):1831,(15,53):2192,(15,54):735,(15,55):1842,(15,56):1071,(15,57):997,(15,58):579,(15,59):534,(15,60):2105,(15,61):1967,(15,62):1876,(15,63):1880,(15,64):484,(15,65):1474,(15,66):1500,(15,67):1893,(15,68):515,(15,69):699,(15,70):674,(15,71):912,(15,72):2111,(15,73):1659,(15,74):1007,(15,75):1040,(15,76):1003,(15,78):569,(15,79):1885,(15,80):222,(15,81):457,(15,82):1871,(15,83):507,(15,84):221,(15,85):684,(15,86):1118,(15,87):541,(15,88):448,(15,89):455,(15,90):404,(15,91):1051,(15,92):1882,(15,93):902,(15,94):1672,(15,95):433,(15,96):640,(15,97):163,(15,98):1893,(15,99):1905,(15,100):377,(15,101):812,(15,102):923,(15,103):476,(15,104):656,(15,105):8,(15,106):1471,(15,107):486,(15,108):410,(15,109):1016,(15,110):665,(15,111):834,(15,112):376,(15,113):2039,(15,114):330,(15,115):1828,(15,116):559,(15,117):228,(15,119):404,(15,121):1124,(15,118):1112,(15,120):898,(15,122):915,(15,123):1667,(15,124):2118,(15,125):1846,(15,126):505,(15,127):639,(15,128):1853,(15,129):1870,(15,130):796,(15,131):441,(15,132):1957,(15,133):1877,(15,134):1667,(15,135):565,(15,136):1886,(15,137):1118,(15,138):995,(15,140):1037,(15,141):1491,(15,142):777,(15,143):487,(15,144):1862,(15,146):918,(15,145):2187,(15,147):1301,(15,148):750,(15,150):1962,(15,151):842,(15,152):1840,(15,153):1020,(15,154):592,(15,155):2067,(15,156):294,(15,157):831,(15,158):779,(15,159):2198,(15,160):2088,(15,161):1484,(15,139):774,(15,162):744,(15,163):1830,(15,164):1842,(15,165):2133,(15,166):2097,(15,167):1875,(15,168):1907,(15,169):2145,(15,170):646,(15,171):1476,(15,172):2210,(15,173):334,(15,174):814,(15,175):1954,(15,178):1674,(15,176):282,(15,177):1246,(15,179):2058,(15,180):2108,(15,181):1661,(15,182):1087,(15,183):2208,(15,184):537,(15,185):774,(15,186):1484,(15,187):675,(15,188):1092,(15,189):1904,(15,190):1437,(15,191):406,(15,192):2187,(15,193):926,(15,194):1940,(15,195):748,(15,196):896,(15,197):531,(15,198):646,(15,199):1291,(15,200):1136,(16,1):767,(16,2):1154,(16,3):1099,(16,4):957,(16,5):1784,(16,8):1096,(16,7):1030,(16,9):1069,(16,10):1175,(16,12):394,(16,11):811,(16,13):1229,(16,14):1773,(16,15):1105,(16,16):1192,(16,17):1630,(16,18):1017,(16,19):1339,(16,20):1324,(16,21):1230,(16,6):1115,(16,22):1576,(16,23):861,(16,24):1666,(16,25):1003,(16,26):1166,(16,27):263,(16,28):1506,(16,29):1109,(16,30):948,(16,31):1241,(16,32):418,(16,33):1803,(16,34):711,(16,35):744,(16,36):855,(16,37):1180,(16,38):738,(16,39):1146,(16,77):1275,(16,40):1766,(16,41):1431,(16,42):1017,(16,43):690,(16,149):1364,(16,44):989,(16,45):816,(16,46):278,(16,47):635,(16,48):1150,(16,49):1791,(16,51):1357,(16,50):1781,(16,52):1777,(16,53):1725,(16,54):1203,(16,55):1749,(16,56):791,(16,57):1759,(16,58):579,(16,59):1029,(16,60):1849,(16,61):1755,(16,62):1785,(16,63):1792,(16,64):1006,(16,65):1499,(16,66):1506,(16,67):1788,(16,68):1029,(16,69):475,(16,70):1101,(16,71):1224,(16,72):1851,(16,73):1524,(16,74):1775,(16,75):831,(16,76):1767,(16,78):1247,(16,79):1799,(16,80):933,(16,81):602,(16,82):1789,(16,83):1015,(16,84):1049,(16,85):1406,(16,86):1195,(16,87):415,(16,88):484,(16,89):485,(16,90):933,(16,91):831,(16,92):1759,(16,93):1478,(16,94):1527,(16,95):779,(16,96):395,(16,97):838,(16,98):1802,(16,99):1800,(16,100):716,(16,101):1074,(16,102):1563,(16,103):983,(16,104):275,(16,105):829,(16,106):1493,(16,107):1017,(16,108):1236,(16,109):1783,(16,110):347,(16,111):2,(16,112):1211,(16,113):1787,(16,114):1120,(16,115):1744,(16,116):401,(16,117):822,(16,119):1233,(16,121):1203,(16,118):1189,(16,120):1211,(16,122):1233,(16,123):1520,(16,124):1853,(16,125):1785,(16,126):818,(16,127):344,(16,128):1760,(16,129):1783,(16,130):1547,(16,131):502,(16,132):1838,(16,133):1758,(16,134):1528,(16,135):1254,(16,136):1783,(16,137):1183,(16,138):1762,(16,140):1159,(16,141):1502,(16,142):876,(16,143):996,(16,144):1767,(16,146):1669,(16,145):1676,(16,147):1314,(16,148):1171,(16,150):1652,(16,151):1139,(16,152):1754,(16,153):926,(16,154):332,(16,155):1780,(16,156):548,(16,157):10,(16,158):1563,(16,159):1705,(16,160):1863,(16,161):1162,(16,139):1194,(16,162):1309,(16,163):1738,(16,164):1798,(16,165):1866,(16,166):1850,(16,167):1790,(16,168):1790,(16,169):1855,(16,170):1306,(16,171):1484,(16,172):1640,(16,173):1020,(16,174):233,(16,175):1369,(16,178):1533,(16,176):634,(16,177):1242,(16,179):1793,(16,180):1857,(16,181):1519,(16,182):1014,(16,183):1649,(16,184):1315,(16,185):1553,(16,186):1503,(16,187):634,(16,188):1215,(16,189):1810,(16,190):1524,(16,191):739,(16,192):1674,(16,193):1252,(16,194):1747,(16,195):635,(16,196):1099,(16,197):644,(16,198):1080,(16,199):1280,(16,200):1199,(17,1):521,(17,2):1314,(17,3):661,(17,4):997,(17,5):2074,(17,7):750,(17,8):662,(17,9):251,(17,10):388,(17,11):1187,(17,12):466,(17,13):886,(17,14):2121,(17,15):702,(17,16):551,(17,17):2314,(17,18):214,(17,19):1926,(17,20):1111,(17,21):954,(17,6):1065,(17,22):1118,(17,23):739,(17,24):843,(17,25):726,(17,26):526,(17,27):565,(17,28):1691,(17,29):401,(17,30):134,(17,31):686,(17,32):465,(17,33):2046,(17,34):283,(17,35):540,(17,36):1180,(17,37):428,(17,38):394,(17,39):324,(17,77):1112,(17,40):2054,(17,41):1000,(17,42):724,(17,43):337,(17,149):1113,(17,44):733,(17,45):1196,(17,46):618,(17,47):556,(17,48):517,(17,49):2090,(17,51):1371,(17,50):2236,(17,52):2037,(17,53):2332,(17,54):528,(17,55):2043,(17,56):1219,(17,57):937,(17,58):413,(17,59):761,(17,60):2285,(17,61):2152,(17,62):2078,(17,63):2082,(17,64):712,(17,65):1683,(17,66):1707,(17,67):2093,(17,68):743,(17,69):558,(17,70):456,(17,71):689,(17,72):2291,(17,73):1851,(17,74):952,(17,75):1200,(17,76):945,(17,78):785,(17,79):2087,(17,80):118,(17,81):297,(17,82):2074,(17,83):735,(17,84):389,(17,85):589,(17,86):890,(17,87):445,(17,88):556,(17,89):566,(17,90):186,(17,91):1210,(17,92):2079,(17,93):1124,(17,94):1863,(17,95):214,(17,96):736,(17,97):388,(17,98):2095,(17,99):2105,(17,100):180,(17,101):1030,(17,102):1136,(17,103):703,(17,104):586,(17,105):230,(17,106):1680,(17,107):715,(17,108):553,(17,109):961,(17,110):566,(17,111):821,(17,112):455,(17,113):2219,(17,114):312,(17,115):2029,(17,116):464,(17,117):0,(17,119):541,(17,121):896,(17,118):884,(17,120):674,(17,122):691,(17,123):1857,(17,124):2297,(17,125):2050,(17,126):711,(17,127):718,(17,128):2053,(17,129):2072,(17,130):726,(17,131):557,(17,132):2155,(17,133):2074,(17,134):1859,(17,135):779,(17,136):2086,(17,137):890,(17,138):940,(17,140):808,(17,141):1699,(17,142):556,(17,143):714,(17,144):2063,(17,146):849,(17,145):2317,(17,147):1073,(17,148):535,(17,150):2131,(17,151):616,(17,152):2042,(17,153):808,(17,154):517,(17,155):2241,(17,156):301,(17,157):815,(17,158):741,(17,159):2332,(17,160):2273,(17,161):1641,(17,139):1002,(17,162):970,(17,163):2031,(17,164):2048,(17,165):2311,(17,166):2278,(17,167):2077,(17,168):2106,(17,169):2320,(17,170):508,(17,171):1683,(17,172):2325,(17,173):560,(17,174):864,(17,175):2061,(17,178):1866,(17,176):433,(17,177):1020,(17,179):2237,(17,180):2289,(17,181):1852,(17,182):870,(17,183):2325,(17,184):494,(17,185):730,(17,186):1693,(17,187):491,(17,188):864,(17,189):2106,(17,190):1652,(17,191):601,(17,192):2316,(17,193):704,(17,194):2128,(17,195):566,(17,196):668,(17,197):698,(17,198):428,(17,199):1065,(17,200):908,(18,1):1067,(18,2):1209,(18,3):1131,(18,4):911,(18,5):1942,(18,8):1133,(18,7):538,(18,9):499,(18,10):628,(18,11):1258,(18,12):969,(18,13):541,(18,14):2024,(18,15):1178,(18,16):84,(18,17):2465,(18,18):369,(18,19):2023,(18,20):1602,(18,21):1440,(18,6):885,(18,22):625,(18,23):1282,(18,24):639,(18,25):534,(18,26):904,(18,27):984,(18,28):1533,(18,29):764,(18,30):531,(18,31):1083,(18,32):980,(18,33):1884,(18,34):835,(18,35):1089,(18,36):1222,(18,37):716,(18,38):943,(18,39):418,(18,77):1618,(18,40):1921,(18,41):546,(18,42):519,(18,43):891,(18,149):767,(18,44):556,(18,45):1268,(18,46):975,(18,47):1110,(18,48):906,(18,49):1962,(18,51):1153,(18,50):2206,(18,52):1886,(18,53):2413,(18,54):872,(18,55):1915,(18,56):1316,(18,57):778,(18,58):962,(18,59):554,(18,60):2236,(18,61):2087,(18,62):1946,(18,63):1948,(18,64):515,(18,65):1525,(18,66):1557,(18,67):1968,(18,68):531,(18,69):1097,(18,70):858,(18,71):1098,(18,72):2244,(18,73):1767,(18,74):778,(18,75):1263,(18,76):781,(18,78):375,(18,79):1951,(18,80):454,(18,81):842,(18,82):1937,(18,83):534,(18,84):192,(18,85):593,(18,86):1374,(18,87):951,(18,88):804,(18,89):808,(18,90):644,(18,91):1277,(18,92):1966,(18,93):700,(18,94):1784,(18,95):759,(18,96):987,(18,97):418,(18,98):1961,(18,99):1979,(18,100):733,(18,101):868,(18,102):660,(18,103):527,(18,104):1070,(18,105):419,(18,106):1525,(18,107):508,(18,108):8,(18,109):787,(18,110):1076,(18,111):1239,(18,112):151,(18,113):2173,(18,114):328,(18,115):1897,(18,116):969,(18,117):553,(18,119):13,(18,121):1379,(18,118):1369,(18,120):1087,(18,122):1096,(18,123):1780,(18,124):2253,(18,125):1903,(18,126):678,(18,127):1000,(18,128):1925,(18,129):1938,(18,130):630,(18,131):792,(18,132):2033,(18,133):1959,(18,134):1776,(18,135):359,(18,136):1961,(18,137):1379,(18,138):770,(18,140):1287,(18,141):1548,(18,142):1085,(18,143):527,(18,144):1935,(18,146):728,(18,145):2428,(18,147):1559,(18,148):914,(18,150):2126,(18,151):1052,(18,152):1910,(18,153):1347,(18,154):1005,(18,155):2214,(18,156):707,(18,157):1236,(18,158):559,(18,159):2430,(18,160):2206,(18,161):1691,(18,139):737,(18,162):597,(18,163):1903,(18,164):1891,(18,165):2267,(18,166):2225,(18,167):1942,(18,168):1987,(18,169):2290,(18,170):646,(18,171):1535,(18,172):2475,(18,173):330,(18,174):1187,(18,175):2236,(18,178):1784,(18,176):629,(18,177):1516,(18,179):2197,(18,180):2237,(18,181):1773,(18,182):1399,(18,183):2469,(18,184):410,(18,185):565,(18,186):1537,(18,187):1044,(18,188):1334,(18,189):1973,(18,190):1461,(18,191):641,(18,192):2429,(18,193):1101,(18,194):2052,(18,195):1120,(18,196):1138,(18,197):798,(18,198):836,(18,199):1559,(18,200):1395,(19,1):2063,(19,2):391,(19,3):2328,(19,4):703,(19,5):394,(19,7):1020,(19,8):2327,(19,9):1870,(19,10):2037,(19,11):699,(19,12):1670,(19,13):1022,(19,14):491,(19,15):2358,(19,16):1465,(19,17):1308,(19,18):1712,(19,19):868,(19,20):2708,(19,21):2573,(19,6):667,(19,22):1198,(19,23):2244,(19,24):2183,(19,25):1033,(19,26):2228,(19,27):1481,(19,28):18,(19,29):2098,(19,30):1804,(19,31):2385,(19,32):1707,(19,33):346,(19,34):1860,(19,35):2058,(19,36):651,(19,37):2104,(19,38):1962,(19,39):1855,(19,77):2680,(19,40):374,(19,41):1135,(19,42):1044,(19,43):1885,(19,149):844,(19,44):1016,(19,45):697,(19,46):1359,(19,47):1986,(19,48):2219,(19,49):415,(19,51):407,(19,50):733,(19,52):341,(19,53):1110,(19,54):2227,(19,55):368,(19,56):736,(19,57):2324,(19,58):1844,(19,59):1006,(19,60):737,(19,61):581,(19,62):398,(19,63):400,(19,64):1052,(19,65):23,(19,66):13,(19,67):422,(19,68):1028,(19,69):1849,(19,70):2158,(19,71):2384,(19,72):746,(19,73):287,(19,74):2322,(19,75):680,(19,76):2326,(19,78):1193,(19,79):403,(19,80):1715,(19,81):1764,(19,82):389,(19,83):1030,(19,84):1495,(19,85):2111,(19,86):2519,(19,87):1675,(19,88):1237,(19,89):1226,(19,90):1887,(19,91):682,(19,92):424,(19,93):1005,(19,94):308,(19,95):1856,(19,96):1162,(19,97):1332,(19,98):414,(19,99):432,(19,100):1775,(19,101):691,(19,102):1139,(19,103):1048,(19,104):1641,(19,105):1488,(19,106):22,(19,107):1056,(19,108):1539,(19,109):2331,(19,110):1720,(19,111):1506,(19,112):1667,(19,113):683,(19,114):1761,(19,115):350,(19,116):1678,(19,117):1701,(19,119):1555,(19,121):2526,(19,118):2512,(19,120):2369,(19,122):2388,(19,123):308,(19,124):758,(19,125):357,(19,126):990,(19,127):1216,(19,128):378,(19,129):391,(19,130):2173,(19,131):1224,(19,132):487,(19,133):417,(19,134):296,(19,135):1212,(19,136):415,(19,137):2513,(19,138):2315,(19,140):2452,(19,141):7,(19,142):2152,(19,143):1043,(19,144):388,(19,146):2276,(19,145):1186,(19,147):2683,(19,148):2237,(19,150):709,(19,151):2305,(19,152):363,(19,153):2320,(19,154):1638,(19,155):746,(19,156):1494,(19,157):1514,(19,158):2107,(19,159):1160,(19,160):689,(19,161):597,(19,139):814,(19,162):995,(19,163):357,(19,164):349,(19,165):771,(19,166):720,(19,167):395,(19,168):442,(19,169):810,(19,170):2115,(19,171):23,(19,172):1315,(19,173):1249,(19,174):1273,(19,175):1203,(19,178):303,(19,176):1282,(19,177):2615,(19,179):713,(19,180):734,(19,181):299,(19,182):2406,(19,183):1293,(19,184):1930,(19,185):2112,(19,186):10,(19,187):1946,(19,188):2511,(19,189):426,(19,190):129,(19,191):1100,(19,192):1191,(19,193):2402,(19,194):538,(19,195):1992,(19,196):2332,(19,197):1033,(19,198):2130,(19,199):2660,(19,200):2531,(20,1):108,(20,2):1763,(20,3):224,(20,4):1466,(20,5):2500,(20,8):221,(20,7):1287,(20,9):619,(20,10):586,(20,12):522,(20,11):1540,(20,13):1438,(20,14):2523,(20,15):229,(20,16):1101,(20,17):2506,(20,18):719,(20,19):2192,(20,20):566,(20,21):420,(20,6):1566,(20,22):1676,(20,23):207,(20,24):1135,(20,25):1260,(20,26):389,(20,27):727,(20,28):2147,(20,29):433,(20,30):557,(20,31):386,(20,32):486,(20,33):2490,(20,34):305,(20,35):134,(20,36):1556,(20,37):527,(20,38):192,(20,39):724,(20,77):557,(20,40):2480,(20,41):1559,(20,42):1262,(20,43):270,(20,149):1659,(20,44):1263,(20,45):1548,(20,46):843,(20,47):246,(20,48):372,(20,49):2512,(20,51):1869,(20,50):2587,(20,52):2474,(20,53):2588,(20,54):445,(20,55):2466,(20,56):1550,(20,57):1167,(20,58):323,(20,59):1296,(20,60):2647,(20,61):2531,(20,62):2502,(20,63):2508,(20,64):1248,(20,65):2139,(20,66):2158,(20,67):2513,(20,68):1280,(20,69):401,(20,70):358,(20,71):363,(20,72):2651,(20,73):2256,(20,74):1190,(20,75):1557,(20,76):1176,(20,78):1344,(20,79):2514,(20,80):637,(20,81):389,(20,82):2502,(20,83):1270,(20,84):944,(20,85):834,(20,86):365,(20,87):509,(20,88):919,(20,89):930,(20,90):447,(20,91):1563,(20,92):2492,(20,93):1682,(20,94):2264,(20,95):346,(20,96):1033,(20,97):926,(20,98):2520,(20,99):2525,(20,100):408,(20,101):1527,(20,102):1695,(20,103):1236,(20,104):610,(20,105):780,(20,106):2135,(20,107):1253,(20,108):1089,(20,109):1197,(20,110):530,(20,111):875,(20,112):962,(20,113):2582,(20,114):780,(20,115):2456,(20,116):513,(20,117):559,(20,119):1074,(20,121):372,(20,118):358,(20,120):352,(20,122):373,(20,123):2258,(20,124):2655,(20,125):2486,(20,126):1196,(20,127):989,(20,128):2477,(20,129):2498,(20,130):980,(20,131):930,(20,132):2571,(20,133):2489,(20,134):2262,(20,135):1339,(20,136):2506,(20,137):359,(20,138):1178,(20,140):302,(20,141):2151,(20,142):3,(20,143):1249,(20,144):2486,(20,146):1075,(20,145):2546,(20,147):534,(20,148):388,(20,150):2467,(20,151):286,(20,152):2468,(20,153):261,(20,154):575,(20,155):2589,(20,156):662,(20,157):865,(20,158):1051,(20,159):2573,(20,160):2648,(20,161):1966,(20,139):1534,(20,162):1524,(20,163):2454,(20,164):2490,(20,165):2669,(20,166):2644,(20,167):2504,(20,168):2521,(20,169):2667,(20,170):693,(20,171):2133,(20,172):2516,(20,173):1114,(20,174):1043,(20,175):2245,(20,178):2268,(20,176):886,(20,177):473,(20,179):2594,(20,180):2653,(20,181):2254,(20,182):312,(20,183):2524,(20,184):864,(20,185):1033,(20,186):2147,(20,187):242,(20,188):362,(20,189):2529,(20,190):2126,(20,191):1078,(20,192):2544,(20,193):393,(20,194):2514,(20,195):249,(20,196):223,(20,197):1120,(20,198):356,(20,199):518,(20,200):377,(21,1):469,(21,2):1851,(21,3):267,(21,4):1535,(21,5):2611,(21,7):1268,(21,8):271,(21,9):420,(21,10):293,(21,11):1705,(21,12):783,(21,13):1376,(21,14):2656,(21,15):315,(21,16):955,(21,17):2775,(21,18):576,(21,19):2419,(21,20):719,(21,21):560,(21,6):1597,(21,22):1538,(21,23):579,(21,24):780,(21,25):1247,(21,26):10,(21,27):967,(21,28):2229,(21,29):152,(21,30):443,(21,31):169,(21,32):756,(21,33):2584,(21,34):468,(21,35):503,(21,36):1705,(21,37):206,(21,38):443,(21,39):497,(21,77):753,(21,40):2591,(21,41):1446,(21,42):1242,(21,43):484,(21,149):1611,(21,44):1257,(21,45):1714,(21,46):1062,(21,47):607,(21,48):24,(21,49):2627,(21,51):1902,(21,50):2760,(21,52):2574,(21,53):2823,(21,54):60,(21,55):2579,(21,56):1730,(21,57):791,(21,58):598,(21,59):1281,(21,60):2812,(21,61):2683,(21,62):2615,(21,63):2619,(21,64):1231,(21,65):2221,(21,66):2245,(21,67):2629,(21,68):1261,(21,69):733,(21,70):82,(21,71):185,(21,72):2817,(21,73):2385,(21,74):816,(21,75):1719,(21,76):800,(21,78):1245,(21,79):2624,(21,80):538,(21,81):583,(21,82):2611,(21,83):1254,(21,84):829,(21,85):489,(21,86):497,(21,87):763,(21,88):1063,(21,89):1074,(21,90):351,(21,91):1729,(21,92):2615,(21,93):1590,(21,94):2397,(21,95):421,(21,96):1222,(21,97):915,(21,98):2631,(21,99):2641,(21,100):503,(21,101):1564,(21,102):1570,(21,103):1226,(21,104):900,(21,105):756,(21,106):2218,(21,107):1232,(21,108):918,(21,109):821,(21,110):834,(21,111):1173,(21,112):765,(21,113):2746,(21,114):586,(21,115):2566,(21,116):775,(21,117):537,(21,119):900,(21,121):499,(21,118):493,(21,120):175,(21,122):182,(21,123):2391,(21,124):2823,(21,125):2588,(21,126):1248,(21,127):1191,(21,128):2590,(21,129):2609,(21,130):621,(21,131):1068,(21,132):2692,(21,133):2610,(21,134):2393,(21,135):1234,(21,136):2623,(21,137):506,(21,138):804,(21,140):412,(21,141):2237,(21,142):389,(21,143):1235,(21,144):2600,(21,146):702,(21,145):2796,(21,147):672,(21,148):4,(21,150):2651,(21,151):162,(21,152):2579,(21,153):605,(21,154):844,(21,155):2765,(21,156):787,(21,157):1165,(21,158):709,(21,159):2817,(21,160):2803,(21,161):2154,(21,139):1519,(21,162):1454,(21,163):2567,(21,164):2586,(21,165):2837,(21,166):2806,(21,167):2614,(21,168):2642,(21,169):2844,(21,170):349,(21,171):2221,(21,172):2786,(21,173):1056,(21,174):1295,(21,175):2517,(21,178):2400,(21,176):966,(21,177):644,(21,179):2762,(21,180):2816,(21,181):2386,(21,182):612,(21,183):2790,(21,184):579,(21,185):689,(21,186):2231,(21,187):573,(21,188):444,(21,189):2642,(21,190):2188,(21,191):1138,(21,192):2795,(21,193):187,(21,194):2660,(21,195):613,(21,196):276,(21,197):1229,(21,198):110,(21,199):682,(21,200):519,(22,1):2363,(22,2):925,(22,3):2673,(22,4):1214,(22,5):480,(22,7):1588,(22,8):2671,(22,9):2345,(22,10):2507,(22,11):945,(22,12):1946,(22,13):1647,(22,14):343,(22,15):2692,(22,16):2041,(22,17):672,(22,18):2202,(22,19):397,(22,20):2970,(22,21):2862,(22,6):1258,(22,22):1880,(22,23):2500,(22,24):2761,(22,25):1591,(22,26):2640,(22,27):1740,(22,28):726,(22,29):2528,(22,30):2254,(22,31):2771,(22,32):1982,(22,33):590,(22,34):2210,(22,35):2348,(22,36):950,(22,37):2558,(22,38):2289,(22,39):2356,(22,77):2925,(22,40):483,(22,41):1798,(22,42):1606,(22,43):2218,(22,149):1511,(22,44):1569,(22,45):936,(22,46):1623,(22,47):2253,(22,48):2627,(22,49):460,(22,51):1081,(22,50):134,(22,52):550,(22,53):419,(22,54):2653,(22,55):467,(22,56):922,(22,57):2898,(22,58):2143,(22,59):1573,(22,60):214,(22,61):223,(22,62):476,(22,63):483,(22,64):1610,(22,65):729,(22,66):694,(22,67):447,(22,68):1594,(22,69):2096,(22,70):2567,(22,71):2763,(22,72):212,(22,73):420,(22,74):2900,(22,75):931,(22,76):2901,(22,78):1811,(22,79):489,(22,80):2178,(22,81):2099,(22,82):495,(22,83):1591,(22,84):2028,(22,85):2634,(22,86):2819,(22,87):1958,(22,88):1587,(22,89):1576,(22,90):2312,(22,91):921,(22,92):408,(22,93):1684,(22,94):400,(22,95):2233,(22,96):1439,(22,97):1818,(22,98):479,(22,99):450,(22,100):2150,(22,101):1266,(22,102):1823,(22,103):1599,(22,104):1873,(22,105):1954,(22,106):725,(22,107):1617,(22,108):2117,(22,109):2909,(22,110):1958,(22,111):1652,(22,112):2220,(22,113):169,(22,114):2273,(22,115):486,(22,116):1954,(22,117):2130,(22,119):2130,(22,121):2827,(22,118):2812,(22,120):2748,(22,122):2769,(22,123):400,(22,124):209,(22,125):538,(22,126):1474,(22,127):1478,(22,128):468,(22,129):483,(22,130):2723,(22,131):1581,(22,132):435,(22,133):415,(22,134):411,(22,135):1830,(22,136):450,(22,137):2809,(22,138):2891,(22,140):2768,(22,141):702,(22,142):2466,(22,143):1598,(22,144):465,(22,146):2836,(22,145):518,(22,147):2956,(22,148):2647,(22,150):1,(22,151):2678,(22,152):481,(22,153):2570,(22,154):1896,(22,155):130,(22,156):1863,(22,157):1662,(22,158):2674,(22,159):480,(22,160):259,(22,161):501,(22,139):1438,(22,162):1642,(22,163):468,(22,164):574,(22,165):220,(22,166):223,(22,167):488,(22,168):425,(22,169):202,(22,170):2607,(22,171):703,(22,172):676,(22,173):1800,(22,174):1440,(22,175):678,(22,178):404,(22,176):1698,(22,177):2884,(22,179):158,(22,180):224,(22,181):409,(22,182):2658,(22,183):648,(22,184):2465,(22,185):2676,(22,186):718,(22,187):2231,(22,188):2827,(22,189):473,(22,190):837,(22,191):1557,(22,192):525,(22,193):2786,(22,194):262,(22,195):2256,(22,196):2675,(22,197):1434,(22,198):2540,(22,199):2925,(22,200):2826,(23,1):2564,(23,2):1056,(23,3):2869,(23,4):1362,(23,5):439,(23,8):2867,(23,7):1727,(23,9):2515,(23,10):2679,(23,11):1133,(23,12):2147,(23,13):1766,(23,14):298,(23,15):2890,(23,16):2181,(23,17):800,(23,18):2368,(23,19):609,(23,20):3177,(23,21):3065,(23,6):1385,(23,22):1968,(23,23):2706,(23,24):2904,(23,25):1733,(23,26):2825,(23,27):1941,(23,28):788,(23,29):2709,(23,30):2429,(23,31):2961,(23,32):2183,(23,33):551,(23,34):2403,(23,35):2550,(23,36):1130,(23,37):2734,(23,38):2486,(23,39):2520,(23,77):3133,(23,40):451,(23,41):1899,(23,42):1747,(23,43):2414,(23,149):1608,(23,44):1712,(23,45):1124,(23,46):1823,(23,47):2457,(23,48):2813,(23,49):414,(23,51):1170,(23,50):84,(23,52):518,(23,53):512,(23,54):2835,(23,55):443,(23,56):1116,(23,57):3043,(23,58):2343,(23,59):1712,(23,60):33,(23,61):188,(23,62):434,(23,63):439,(23,64):1753,(23,65):793,(23,66):757,(23,67):401,(23,68):1734,(23,69):2301,(23,70):2753,(23,71):2954,(23,72):23,(23,73):503,(23,74):3044,(23,75):1117,(23,76):3046,(23,78):1934,(23,79):442,(23,80):2350,(23,81):2293,(23,82):453,(23,83):1732,(23,84):2182,(23,85):2794,(23,86):3021,(23,87):2158,(23,88):1774,(23,89):1763,(23,90):2493,(23,91):1109,(23,92):375,(23,93):1773,(23,94):485,(23,95):2422,(23,96):1634,(23,97):1982,(23,98):429,(23,99):399,(23,100):2339,(23,101):1398,(23,102):1909,(23,103):1743,(23,104):2079,(23,105):2124,(23,106):790,(23,107):1758,(23,108):2258,(23,109):3053,(23,110):2163,(23,111):1863,(23,112):2369,(23,113):92,(23,114):2434,(23,115):465,(23,116):2155,(23,117):2309,(23,119):2271,(23,121):3029,(23,118):3015,(23,120):2940,(23,122):2960,(23,123):488,(23,124):12,(23,125):502,(23,126):1634,(23,127):1677,(23,128):439,(23,129):443,(23,130):2875,(23,131):1767,(23,132):360,(23,133):384,(23,134):494,(23,135):1954,(23,136):408,(23,137):3012,(23,138):3035,(23,140):2968,(23,141):766,(23,142):2665,(23,143):1741,(23,144):431,(23,146):2986,(23,145):632,(23,147):3162,(23,148):2833,(23,150):218,(23,151):2870,(23,152):455,(23,153):2777,(23,154):2099,(23,155):88,(23,156):2050,(23,157):1873,(23,158):2821,(23,159):582,(23,160):102,(23,161):704,(23,139):1555,(23,162):1751,(23,163):449,(23,164):535,(23,165):2,(23,166):53,(23,167):445,(23,168):376,(23,169):57,(23,170):2775,(23,171):772,(23,172):801,(23,173):1947,(23,174):1648,(23,175):859,(23,178):486,(23,176):1875,(23,177):3090,(23,179):72,(23,180):40,(23,181):496,(23,182):2865,(23,183):771,(23,184):2621,(23,185):2824,(23,186):780,(23,187):2433,(23,188):3028,(23,189):419,(23,190):892,(23,191):1724,(23,192):639,(23,193):2977,(23,194):233,(23,195):2460,(23,196):2871,(23,197):1610,(23,198):2725,(23,199):3132,(23,200):3029,(24,1):2407,(24,2):1394,(24,3):2738,(24,4):1601,(24,5):1154,(24,7):1969,(24,8):2735,(24,9):2566,(24,10):2712,(24,11):1216,(24,12):2010,(24,13):2083,(24,14):1018,(24,15):2745,(24,16):2393,(24,17):9,(24,18):2450,(24,19):476,(24,20):2933,(24,21):2858,(24,6):1702,(24,22):2377,(24,23):2489,(24,24):3076,(24,25):1963,(24,26):2775,(24,27):1818,(24,28):1331,(24,29):2691,(24,30):2457,(24,31):2872,(24,32):2042,(24,33):1263,(24,34):2317,(24,35):2384,(24,36):1258,(24,37):2743,(24,38):2367,(24,39):2605,(24,77):2873,(24,40):1156,(24,41):2269,(24,42):1981,(24,43):2309,(24,149):2006,(24,44):1939,(24,45):1206,(24,46):1725,(24,47):2274,(24,48):2760,(24,49):1135,(24,51):1618,(24,50):752,(24,52):1222,(24,53):293,(24,54):2803,(24,55):1140,(24,56):1158,(24,57):3202,(24,58):2208,(24,59):1956,(24,60):821,(24,61):895,(24,62):1150,(24,63):1158,(24,64):1981,(24,65):1332,(24,66):1301,(24,67):1122,(24,68):1975,(24,69):2115,(24,70):2705,(24,71):2857,(24,72):813,(24,73):1042,(24,74):3209,(24,75):1212,(24,76):3207,(24,78):2226,(24,79):1164,(24,80):2401,(24,81):2202,(24,82):1170,(24,83):1967,(24,84):2336,(24,85):2890,(24,86):2829,(24,87):2028,(24,88):1772,(24,89):1763,(24,90):2486,(24,91):1198,(24,92):1082,(24,93):2186,(24,94):1023,(24,95):2366,(24,96):1587,(24,97):2098,(24,98):1154,(24,99):1125,(24,100):2288,(24,101):1695,(24,102):2325,(24,103):1963,(24,104):1907,(24,105):2201,(24,106):1326,(24,107):1990,(24,108):2466,(24,109):3219,(24,110):1985,(24,111):1640,(24,112):2534,(24,113):814,(24,114):2539,(24,115):1157,(24,116):2019,(24,117):2323,(24,119):2475,(24,121):2837,(24,118):2823,(24,120):2843,(24,122):2865,(24,123):1020,(24,124):801,(24,125):1211,(24,126):1795,(24,127):1607,(24,128):1142,(24,129):1157,(24,130):3005,(24,131):1775,(24,132):1109,(24,133):1090,(24,134):1035,(24,135):2244,(24,136):1125,(24,137):2816,(24,138):3199,(24,140):2798,(24,141):1307,(24,142):2515,(24,143):1967,(24,144):1138,(24,146):3126,(24,145):169,(24,147):2931,(24,148):2781,(24,150):674,(24,151):2771,(24,152):1154,(24,153):2547,(24,154):1951,(24,155):738,(24,156):2025,(24,157):1650,(24,158):2980,(24,159):222,(24,160):891,(24,161):822,(24,139):1891,(24,162):2103,(24,163):1140,(24,164):1247,(24,165):801,(24,166):840,(24,167):1162,(24,168):1100,(24,169):746,(24,170):2831,(24,171):1302,(24,172):1,(24,173):2145,(24,174):1489,(24,175):270,(24,178):1030,(24,176):1926,(24,177):2859,(24,179):787,(24,180):832,(24,181):1028,(24,182):2632,(24,183):30,(24,184):2745,(24,185):2977,(24,186):1323,(24,187):2273,(24,188):2853,(24,189):1149,(24,190):1441,(24,191):1842,(24,192):162,(24,193):2884,(24,194):936,(24,195):2275,(24,196):2738,(24,197):1691,(24,198):2681,(24,199):2893,(24,200):2830,(25,1):577,(25,2):1106,(25,3):832,(25,4):806,(25,5):1852,(25,7):666,(25,8):831,(25,9):548,(25,10):684,(25,11):921,(25,12):281,(25,13):846,(25,14):1885,(25,15):861,(25,16):669,(25,17):2019,(25,18):471,(25,19):1641,(25,20):1222,(25,21):1080,(25,6):911,(25,22):1149,(25,23):779,(25,24):1125,(25,25):638,(25,26):774,(25,27):287,(25,28):1489,(25,29):670,(25,30):432,(25,31):908,(25,32):304,(25,33):1836,(25,34):362,(25,35):579,(25,36):924,(25,37):715,(25,38):467,(25,39):610,(25,77):1204,(25,40):1832,(25,41):1012,(25,42):644,(25,43):389,(25,149):1044,(25,44):635,(25,45):930,(25,46):321,(25,47):534,(25,48):761,(25,49):1866,(25,51):1211,(25,50):1977,(25,52):1822,(25,53):2046,(25,54):790,(25,55):1819,(25,56):945,(25,57):1228,(25,58):373,(25,59):672,(25,60):2031,(25,61):1905,(25,62):1855,(25,63):1860,(25,64):630,(25,65):1482,(25,66):1501,(25,67):1867,(25,68):661,(25,69):449,(25,70):702,(25,71):902,(25,72):2036,(25,73):1616,(25,74):1242,(25,75):936,(25,76):1235,(25,78):802,(25,79):1865,(25,80):395,(25,81):271,(25,82):1853,(25,83):649,(25,84):514,(25,85):885,(25,86):1024,(25,87):269,(25,88):279,(25,89):290,(25,90):458,(25,91):945,(25,92):1849,(25,93):1103,(25,94):1626,(25,95):369,(25,96):444,(25,97):348,(25,98):1872,(25,99):1879,(25,100):286,(25,101):871,(25,102):1151,(25,103):613,(25,104):365,(25,105):287,(25,106):1477,(25,107):638,(25,108):703,(25,109):1251,(25,110):385,(25,111):548,(25,112):662,(25,113):1965,(25,114):573,(25,115):1808,(25,116):284,(25,117):298,(25,119):697,(25,121):1031,(25,118):1017,(25,120):888,(25,122):908,(25,123):1619,(25,124):2041,(25,125):1834,(25,126):540,(25,127):421,(25,128):1830,(25,129):1850,(25,130):1018,(25,131):285,(25,132):1928,(25,133):1846,(25,134):1623,(25,135):803,(25,136):1861,(25,137):1019,(25,138):1230,(25,140):954,(25,141):1494,(25,142):657,(25,143):627,(25,144):1839,(25,146):1142,(25,145):2026,(25,147):1193,(25,148):781,(25,150):1866,(25,151):820,(25,152):1820,(25,153):856,(25,154):304,(25,155):1981,(25,156):3,(25,157):544,(25,158):1023,(25,159):2044,(25,160):2024,(25,161):1370,(25,139):900,(25,162):933,(25,163):1807,(25,164):1837,(25,165):2055,(25,166):2026,(25,167):1856,(25,168):1877,(25,169):2060,(25,170):807,(25,171):1477,(25,172):2029,(25,173):555,(25,174):566,(25,175):1765,(25,178):1629,(25,176):230,(25,177):1129,(25,179):1980,(25,180):2035,(25,181):1615,(25,182):936,(25,183):2031,(25,184):774,(25,185):1014,(25,186):1490,(25,187):479,(25,188):1014,(25,189):1882,(25,190):1466,(25,191):423,(25,192):2026,(25,193):924,
                (25,194):1884,(25,195):543,(25,196):836,(25,197):464,(25,198):674,(25,199):1175,(25,200):1037}
    return warehouses, customers, customer_demands, distance



In [5]:
# In this function, we are testing the input paramaters
def test_input(warehouses, customers, customer_demands, distance, distance_band, wh_status):
    for c in customers:
        for w in warehouses:
            if(distance[w,c]>=0):
                pass
            else:
                file.write(f'\nDistance between warehouse {w} and customer {c} is not available or invalid')
    
    wh_counter = 0
    for w in wh_status:
        if wh_status[w][1] == 1:
            if wh_status[w][2]==1:
                wh_counter = wh_counter+1
            else:
                print("In wh_status, if the first value is 1, the second value can't be 0.  The problem is WH: ", w)
                file.write(f'\nwh_status for warehouse {w} cannot have a 1 and 0 for status')
    if wh_counter <= number_of_whs:
        pass
    else: 
        print("Your fixed number of warehouses in wh_status is larger than the number of warehouses.  You need to fix and re-run")
        file.write(f'\nYour fixed number of warehouses in wh_status is larger than the number of warehouses.  You need to fix and re-run')                   
        


    for c in customers.keys():
        if(customer_demands[c]>=0):
            pass
        else:
            file.write(f'\nDemand for Customer {c} is not available')
        
    
    if ((distance_band[0] < distance_band[1]) and (distance_band[1]  < distance_band[2]) and (distance_band[2]  < distance_band[3]) ):
        pass
    else:
        file.write(f'\nDistance bands are not in ascending order.  You must fix and re-run *********')
        
    if isinstance(number_of_whs, int)==False:
        file.write(f'\nnumber_of_whs = {number_of_whs} is not integer.  You must fix and re-run***********')
              
    
        

In [6]:
# In this function, we are visualizaing the input data-- the data before the optimization has run
def input_visual(warehouses, customers):
    warehouse_list = []
    for w in warehouses.keys():
        wh = {
                    'text':'Warehouse-'+warehouses[w][1],
                    'State':warehouses[w][2],
                    'ZipCode':warehouses[w][3],
                    'lat':warehouses[w][4],
                    'long':warehouses[w][5],
                    'cnt':10000000,
                    'size' : 30,
                    'color' : 'rgba(0, 100, 0)'
                    }
        warehouse_list.append(wh)    

    customer_list =[] 
    for c in customers.keys():
        cust = {
            'text':'Customer-'+customers[c][1],
            'State':customers[c][2],
            'ZipCode':customers[c][3],
            'lat':customers[c][4],
            'long':customers[c][5] ,
            'cnt':customer_demands[c],
            'size' : 3,
            'color' : 'rgb(255, 0, 0)'
        }
        customer_list.append(cust)       

    df = pd.DataFrame.from_records(warehouse_list)
    df['shape'] =  "triangle-down"
    df_cust = pd.DataFrame.from_records(customer_list)
    df_cust['shape'] =  "circle"
    df = df.append(df_cust, ignore_index = True) 

    locations = [ dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = df['long'],
            lat = df['lat'],
            hoverinfo = 'text',
            text = df['text'],
            mode = 'markers',
            marker = dict( 
                size=df['size'], 
                color=df['color'],
                symbol = df['shape'],
                line = dict(
                    width=3,
                    color='rgba(68, 68, 68, 0)'
                ),

            ))]

    layout = dict(
            title = 'Als Athletics - Input',
            showlegend = False, 
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showland = True,
                landcolor = 'rgb(243, 243, 243)',
                countrycolor = 'rgb(204, 204, 204)',
            ),
        )
    fig = py.graph_objs.Figure(data = locations, layout = layout)
    fig.show()
    #py.offline.plot({ "data":locations, "layout":layout}, filename = scenario_name+'_input.html')  

    

In [7]:
# In this function, we are visualizaing the output data-- the data after the optimization has run
def output_visual(warehouses, customers, df_cu, wh_loc, scenario_name ):   
    
    warehouse_list = []
    for w in wh_loc:
            wh = {
                        'text':'Warehouse-'+warehouses[w][1],
                        'State':warehouses[w][2],
                        'ZipCode':warehouses[w][3],
                        'lat':warehouses[w][4],
                        'long':warehouses[w][5],
                        'cnt':10000000,
                        'size' : 30,
                        'color' : 'rgba(0, 100, 0)'
                        }
            warehouse_list.append(wh)    
            
    customer_list =[] 
    for c in customers.keys():
        cust = {
            'text':'Customer-'+customers[c][1],
            'State':customers[c][2],
            'ZipCode':customers[c][3],
            'lat':customers[c][4],
            'long':customers[c][5] ,
            'cnt':customer_demands[c],
            'size' : 3,
            'color' : 'rgb(255, 0, 0)'
        }
        customer_list.append(cust)
        
    df = pd.DataFrame.from_records(warehouse_list)
    df['shape'] =  "triangle-down"
    df_cust = pd.DataFrame.from_records(customer_list)
    df_cust['shape'] =  "circle"
    df = df.append(df_cust, ignore_index = True) 

    paths = []        
    for i in range( len( df_cu) ):
            paths.append(
                dict(
                    type = 'scattergeo',
                    locationmode = 'USA-states',
                    lon = [ df_cu['Warehouse Longitude'][i], df_cu['Customers Longitude'][i] ],
                    lat = [ df_cu['Warehouse Latitude'][i], df_cu['Customers Latitude'][i] ],
                    mode = 'lines',
                    line = dict(
                        width = 1,
                        color = 'red',
                    ),
                    opacity = 0.8,
                )
            )  
                

    locations = [ dict(
            type = 'scattergeo',
            locationmode = 'USA-states',
            lon = df['long'],
            lat = df['lat'],
            hoverinfo = 'text',
            text = df['text'],
            mode = 'markers',
            marker = dict( 
                size=df['size'], 
                color=df['color'],
                symbol = df['shape'],
                line = dict(
                    width=3,
                    color='rgba(68, 68, 68, 0)'
                ),
            ))]

    layout = dict(
            title = 'Als Athletics - Output',
            showlegend = False, 
            geo = dict(
                scope='usa',
                projection=dict( type='albers usa' ),
                showland = True,
                landcolor = 'rgb(243, 243, 243)',
                countrycolor = 'rgb(204, 204, 204)',
            ),
        )
    fig = py.graph_objs.Figure(data = locations+paths, layout = layout); 
    fig.show(); 
    #py.offline.plot({"data":locations+paths, "layout":layout},filename=scenario_name+'_output.html')         

## This is the block of code that runs all the functions (including the optimization)

In [10]:
#getting input data
warehouses, customers, customer_demands, distance = get_data()

#print("loaded input data")
distance_band = [distance_band_1, distance_band_2, distance_band_3, distance_band_4]

#opening text file in write mode
file = open(scenario_name+'_summary' + '.txt',"w")
file.write('loaded input data')
print("loaded input data")  #Also print to Jupyter

#testing input data
file.write('\nTesting Input Data')
test_input(warehouses, customers, customer_demands, distance, distance_band,wh_status)

#RunCode = ()  #We need some function in here that runs the whole thing.
file.write('\nBuilding the model')
df_assign_vars, list_facility_vars, detailed_dataFrame = optimal_location(number_of_whs, warehouses, customers, customer_demands, distance, distance_band, scenario_name)

#printing the maps
if input_map == False:
    input_visual(warehouses, customers)

if output_map == False:
    output_visual(warehouses, customers, df_assign_vars, list_facility_vars, scenario_name)

#closing the file
file.close()



loaded input data
Optimization Status Optimal
Weighted Average Distance: 164.4 miles
Run Time in seconds 56.1
Percent Demand served within 100 miles : 50.5
Percent Demand served within 400 miles : 90.4
Percent Demand served within 800 miles : 100.0
Percent Demand served within 3200 miles : 100.0


# Analysis

In [293]:

def run_P_Facilities(number_of_whs):
    # getting input data
    warehouses, customers, customer_demands, distance = get_data()
    # print("loaded input data")
    distance_band = [distance_band_1, distance_band_2, distance_band_3, distance_band_4]
    # opening text file in write mode
    #file = open(scenario_name+'summary' + '.txt',"w")
    file.write('loaded input data')
    print("loaded input data")  #Also print to Jupyter
    # testing input data
    file.write('\nTesting Input Data')
    test_input(warehouses, customers, customer_demands, distance, distance_band,wh_status)
    # RunCode = ()  #We need some function in here that runs the whole thing.
    file.write('\nBuilding the model')
    df_assign_vars, list_facility_vars, detailed_dataFrame = optimal_location(number_of_whs, warehouses, customers, customer_demands, distance, distance_band, scenario_name)
    # printing the maps
    if input_map == False:
        input_visual(warehouses, customers)
    if output_map == True:
        output_visual(warehouses, customers, df_assign_vars, list_facility_vars, scenario_name)
    # closing the file
    file.close()
    # Transform
    detailed_dataFrame['Total_Demand'] = detailed_dataFrame['Total Demand to Warehouse'].sum()
    detailed_dataFrame['Porcentage_Demand'] = detailed_dataFrame['Total Demand to Warehouse']/detailed_dataFrame.Total_Demand*100
    detailed_dataFrame.sort_values('Porcentage_Demand', ascending = False, inplace = True)
    detailed_dataFrame.reset_index(drop = True, inplace = True)

    Acum = []
    for i in range(len(detailed_dataFrame.Porcentage_Demand.values)):
        if i == 0:
            Acum.append(detailed_dataFrame.Porcentage_Demand.values[i])
        else:
            Acum.append(detailed_dataFrame.Porcentage_Demand.values[i] + Acum[i-1])
    
    detailed_dataFrame = pd.concat([detailed_dataFrame , pd.Series(Acum)], axis = 1)
    detailed_dataFrame.columns = ['Warehouse Key','Warehouse City','State','ZipCode','Total Demand to Warehouse','Total_Demand','Porcentage_Demand','Acum_Demand']

    return detailed_dataFrame


# Run P = 10 facilities

In [294]:
# opening text file in write mode
file = open(scenario_name+'summary' + '.txt',"w")
DetailedDataFrame_ = run_P_Facilities(10) # Running when P = 10

loaded input data
Optimization Status Optimal
Weighted Average Distance: 164.4 miles
Run Time in seconds 13192.0


In [296]:
DetailedDataFrame_

,Warehouse Key,Warehouse City,State,ZipCode,Total Demand to Warehouse,Total_Demand,Porcentage_Demand,Acum_Demand
0,1,Allentown,PA,18101,15240643.0,78026333.0,19.532692,19.532692
1,8,Dallas,TX,75201,13677157.0,78026333.0,17.528899,37.061590
2,5,Chicago,IL,60602,12784534.0,78026333.0,16.384897,53.446487
3,14,Los Angeles,CA,90001,12624357.0,78026333.0,16.179611,69.626098
4,11,Jacksonville,FL,32201,5161804.0,78026333.0,6.615464,76.241562
5,23,San Francisco,CA,94102,4825549.0,78026333.0,6.184513,82.426075
6,19,Phoenix,AZ,85001,4472823.0,78026333.0,5.732453,88.158528
7,21,Raleigh,NC,27601,4102480.0,78026333.0,5.257815,93.416343
8,9,Denver,CO,80201,2725304.0,78026333.0,3.492800,96.909143
9,24,Seattle,WA,98101,2411682.0,78026333.0,3.090857,100.000000


In [295]:
import plotly.express as px
fig = px.bar(DetailedDataFrame_, x = 'Warehouse City', y = 'Acum_Demand', text = 'Acum_Demand')
fig.show()

In [297]:
DetailedDataFrame_.Acum_Demand.describe()['mean']

71.68185194606032

# Run P = 4 facilities

In [298]:
# opening text file in write mode
file = open(scenario_name+'summary' + '.txt',"w")
DetailedDataFrame_4 = run_P_Facilities(4) # Running when P = 10

loaded input data
Optimization Status Optimal
Weighted Average Distance: 310.1 miles
Run Time in seconds 13344.2


In [299]:
DetailedDataFrame_4

,Warehouse Key,Warehouse City,State,ZipCode,Total Demand to Warehouse,Total_Demand,Porcentage_Demand,Acum_Demand
0,14,Los Angeles,CA,90001,23887405.0,78026333.0,30.614543,30.614543
1,10,Indianapolis,IN,46201,19517935.0,78026333.0,25.014549,55.629091
2,1,Allentown,PA,18101,17923916.0,78026333.0,22.971624,78.600715
3,8,Dallas,TX,75201,16697077.0,78026333.0,21.399285,100.000000


In [300]:
import plotly.express as px
fig = px.bar(DetailedDataFrame_4, x = 'Warehouse City', y = 'Acum_Demand', text = 'Acum_Demand')
fig.show()

# Loop to explore from 1 to 10 facilities and compare

In [15]:
def run_P_Facilities(number_of_whs):
    # getting input data
    warehouses, customers, customer_demands, distance = get_data()
    # print("loaded input data")
    distance_band = [distance_band_1, distance_band_2, distance_band_3, distance_band_4]
    # opening text file in write mode
    #file = open(scenario_name+'summary' + '.txt',"w")
    file.write('loaded input data')
    print("loaded input data")  #Also print to Jupyter
    # testing input data
    file.write('\nTesting Input Data')
    test_input(warehouses, customers, customer_demands, distance, distance_band,wh_status)
    # RunCode = ()  #We need some function in here that runs the whole thing.
    file.write('\nBuilding the model')
    df_assign_vars, list_facility_vars, detailed_dataFrame = optimal_location(number_of_whs, warehouses, customers, customer_demands, distance, distance_band, scenario_name)
    # printing the maps
    if input_map == False:
        input_visual(warehouses, customers)
    if output_map == True:
        output_visual(warehouses, customers, df_assign_vars, list_facility_vars, scenario_name)
    # closing the file
    ######## file.close()
    # Transform
    detailed_dataFrame['Total_Demand'] = detailed_dataFrame['Total Demand to Warehouse'].sum()
    detailed_dataFrame['Porcentage_Demand'] = detailed_dataFrame['Total Demand to Warehouse']/detailed_dataFrame.Total_Demand*100
    detailed_dataFrame.sort_values('Porcentage_Demand', ascending = False, inplace = True)
    detailed_dataFrame.reset_index(drop = True, inplace = True)

    Acum = []
    for i in range(len(detailed_dataFrame.Porcentage_Demand.values)):
        if i == 0:
            Acum.append(detailed_dataFrame.Porcentage_Demand.values[i])
        else:
            Acum.append(detailed_dataFrame.Porcentage_Demand.values[i] + Acum[i-1])
    
    detailed_dataFrame = pd.concat([detailed_dataFrame , pd.Series(Acum)], axis = 1)
    detailed_dataFrame.columns = ['Warehouse Key','Warehouse City','State','ZipCode','Total Demand to Warehouse','Total_Demand','Porcentage_Demand','Acum_Demand']

    print('Mean of acumulative demand = {:.3f}' .format(detailed_dataFrame.Acum_Demand.describe()['mean']))
    
    #print(detailed_dataFrame)

    fig = px.bar(detailed_dataFrame, x = 'Warehouse City', y = 'Acum_Demand', text = 'Acum_Demand'); 
    fig.show(); 

    return detailed_dataFrame


In [18]:
for i in range(10):
    print('Ejecutando P =',str(i+1), 'facilities')
    file = open(scenario_name+'_summary_loop'+ str(i+1) + '.txt',"w")
    run_P_Facilities(i+1)

Ejecutando P = 1 facilities
loaded input data
Optimization Status Optimal
Weighted Average Distance: 1071.0 miles
Run Time in seconds 243.1
Percent Demand served within 100 miles : 1.0
Percent Demand served within 400 miles : 5.1
Percent Demand served within 800 miles : 34.8
Percent Demand served within 3200 miles : 100.0


Mean of acumulative demand = 100.000


Ejecutando P = 2 facilities
loaded input data
Optimization Status Optimal
Weighted Average Distance: 557.3 miles
Run Time in seconds 255.5
Percent Demand served within 100 miles : 12.2
Percent Demand served within 400 miles : 35.4
Percent Demand served within 800 miles : 73.8
Percent Demand served within 3200 miles : 100.0


Mean of acumulative demand = 82.263


Ejecutando P = 3 facilities
loaded input data
Optimization Status Optimal
Weighted Average Distance: 382.9 miles
Run Time in seconds 266.2
Percent Demand served within 100 miles : 18.1
Percent Demand served within 400 miles : 56.0
Percent Demand served within 800 miles : 91.1
Percent Demand served within 3200 miles : 100.0


...

Mean of acumulative demand = 70.523


Ejecutando P = 9 facilities
loaded input data
Optimization Status Optimal
Weighted Average Distance: 177.1 miles
Run Time in seconds 319.0
Percent Demand served within 100 miles : 48.9
Percent Demand served within 400 miles : 87.3
Percent Demand served within 800 miles : 100.0
Percent Demand served within 3200 miles : 100.0


Mean of acumulative demand = 72.564


Ejecutando P = 10 facilities
loaded input data
Optimization Status Optimal
Weighted Average Distance: 164.4 miles
Run Time in seconds 325.1
Percent Demand served within 100 miles : 50.5
Percent Demand served within 400 miles : 90.4
Percent Demand served within 800 miles : 100.0
Percent Demand served within 3200 miles : 100.0


Mean of acumulative demand = 71.682


# Analysis from loop

Como se puede apreciar. En el loop, al correr el modelo para una sola instalación se abriría en Kansas. Donde, ocuparía el 100% de la demanda con una distancia promedio de 1071 millas; la distancia para atender más del 34,8% de la demanda tendría que ser superior a 800 millas e inferior a 3200. Si se abren dos instalaciones, el promedio disminuiria hasta 557,3 millas; abriendo en Cincinnati y Los Angeles, donde, Cincinati ocuparía el 64% de la demanda y Los Angeles el restante; la distancia para atender al menos al 73,8% de la demanda es de 800 millas y menos a 3200. Si se deciden abrir 3 instalaciones, el promedio de la distancia disminuiría a 382,9 millas; abriendo en Baltimore, Los Angeles y Dallas; donde, entre Baltimore y Los Angeles tendrían el 73% de la demanda; con estas 3 instalaciones, se tendría una distancia de 800 millas para atender al 91,1% de la demanda. Si se abrieran 4 instalaciones, el promedio de la distancia disminuye a 310,1 millas. Donde se abriría en: Los Angeles, indianapolis, Allentown y Dallas; donde se atendería cerca del 93% de la demanda a una distancia menor de 800 millas. Sí se abrieran 5 instalaciones el promedio del peso de distancia sería de 265 millas; en este escenario, se atendería cerca del 97% de la demanda con menos de 800 millas; se abrirían las instalaciones en: Los Angeles, Allentown, Dallas, Chicago, Jacksonville; es necesario aclarar que el aporte de esta última instalación sería del 9% para atender a la demanda, dado que, el 91% de la demanda restante es atendido por las otras cuatro instaciones. 

Si se abrieran 6 instalaciones, el peso de la distancia promerio bajaría a 235 millas; se abriría en Allentown, Los Angeles, Dallas, Chicago, San Francisco y Jacksonville y atendería el 98% de la demanda con una distancia de 800 millas, pero el aporte de San Fransisco y Jacksonville representa el 9% y el 8% de la demanda respectivamente. Al abrir cerca de 7 instalaciones, se observa que la distancia promedio ponderada no decrece considerablemente al igual que la demanda atendida con 800 millas de distancia, ya que se atiende cerca del 98% si ordenaramos la demanda de cada instalación y la ponderaramos por la demanda total, tendríamos que la media de este porcentaje acumulado es del 66,8% muy bajo con respecto al 70,1% del escenario de 5 instalaciones. Se abrirían las instalaciones en: Allentown, Dallas, Chicago, Los Angeles, San Francisco, Phoenix, Jacksonville; donde, las últimas 3 instalaciones no atienden más del 10% cada una.

Sí se abrieran 8 instalaciones, con 800 millas se cubriría toda la demanda, además, el promedio de distancia ponderado en la demanda es de 192,8; no obstante, cerca de 3 instalaciones no tendrán una demanda de más de 10% unidas. Si se deciden abrir 9 instalaciones, con 400 millas de distancia se atendería cerca del 87,3% de la demanda y con 800 metros se cubriría el 100% de la demanda; se tendría una distancia promedio ponderada en la demanda de 177,1 millas; no obstante, se tendrían 4 instalaciones que si sumasemos su demanda, no cubrirían más del 13%, es decir, el 87% de la demanda es atendido por las demás instalaciones.Y finalmente, sí se abrierán 10 instalaciones, el promedio ponderado de la distancia con la demanda sería de 164,4 millas; se atendería el 100% de la demanda con 800 millas y el 90% de la demanda con menos de 400 millas; no obstante, 4 de las 10 instalaciones no cubriría más del 12% de la demanda juntas.

# Recomendación
Podríamos concluir entonces, que se debe elegir un número de instalaciones no muy grande, dado que tendríamos instalaciones que aportarían marginalmente al cubrimiento de la demanda. Y así mismo, no podemos elegir abrir pocas instalaciones, dado que la distancia promedio ponderada por la demanda sería alta y no alcanzaría a cubrirse lo suficiente. De este modo, y suponiendo que *no es necesario antender al 100% de la demanda con una distancia menor a 800 millas* se recomienda elegir abrir 4 instalaciones, ubicadas en: Los Angeles, Indianapolis, Allentown, Dallas. Ya que entre ellas, cada una atiende más del 25% de la demanda y se cubre con estas ubicaciones cerca del 93% de la demanda con 800 millas de distancia. Y adicionalmente, la distancia promedio ponderada por la demanda sería de 310 millas.